In [2]:
%load_ext autoreload
%autoreload 2
import sys
from os import listdir
import csv
import pprint
import pickle
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
from plotly import tools
import numpy as np
from numpy import var
from numpy import mean
from math import sqrt
from helpers import functions
from sklearn.metrics import pairwise_distances

plotly.offline.init_notebook_mode(connected=True)
csv.field_size_limit(sys.maxsize)

131072

### Load the data:

In [17]:
bbc_no_dupes_dir = 'data/processed/2019_03_27_BBC_NoDupesWithin.csv'
bbc_no_dupesatall_dir = 'data/processed/2019_03_27_BBC_NoDupesAtAll.csv'
fox_no_dupes_dir = 'data/processed/2019_03_27_FOX_NoDupesWithin.csv'
fox_no_dupesatall_dir = 'data/processed/2019_03_27_FOX_NoDupesAtAll.csv'
cnn_no_dupes_dir = 'data/processed/2019_03_28_CNN_NoDupesWithin.csv'
cnn_no_dupesatall_dir = 'data/processed/2019_03_28_CNN_NoDupesAtAll.csv'
rt_no_dupes_dir = 'data/processed/2019_03_26_RT_NoDupesWithin.csv'
rt_no_dupesatall_dir = 'data/processed/2019_03_26_RT_NoDupesAtAll.csv'

In [18]:
cols_to_keep = ['source',
                'paper_section_name',
                'source_url', 
                'url',
                'canonical_link',
                'rss_link',
                'title',
                'rss_title',
                'text',
                'paper_section_name',
                'pull_date']

BBC:

In [19]:
%%time
df_bbc_clean_nodupes_csv = pd.read_csv(bbc_no_dupes_dir)

CPU times: user 9.61 s, sys: 2.34 s, total: 12 s
Wall time: 12.2 s


In [20]:
%%time
df_bbc_clean_nodupes_atall_csv = pd.read_csv(bbc_no_dupesatall_dir)

CPU times: user 7.47 s, sys: 1.47 s, total: 8.94 s
Wall time: 9.04 s


Fox News:

In [21]:
%%time
df_fox_clean_nodupes_csv = pd.read_csv(fox_no_dupes_dir)

CPU times: user 3.18 s, sys: 487 ms, total: 3.66 s
Wall time: 3.78 s


In [22]:
%%time
df_fox_clean_nodupes_atall_csv = pd.read_csv(fox_no_dupesatall_dir)

CPU times: user 3.32 s, sys: 890 ms, total: 4.21 s
Wall time: 4.3 s


RT:

In [23]:
%%time
df_rt_clean_nodupes_csv = pd.read_csv(rt_no_dupes_dir)

CPU times: user 4.89 s, sys: 1.96 s, total: 6.86 s
Wall time: 7.36 s


In [24]:
%%time
df_rt_clean_nodupes_atall_csv = pd.read_csv(rt_no_dupesatall_dir)

CPU times: user 3.21 s, sys: 598 ms, total: 3.81 s
Wall time: 3.92 s


Loading CNN csv takes too long and they are too bulky. Let's use the 'csv' module instead:

In [25]:
# %%time
# df_cnn_clean_nodupes_csv = pd.read_csv(cnn_no_dupes_dir, names=['text'])

In [26]:
# %%time
# df_cnn_clean_nodupes_atall_csv = pd.read_csv(cnn_no_dupesatall_dir, names=cols_to_keep)

In [27]:
%%time
# CNN texts only
cnn_clean_nodupes_texts = []
with open(cnn_no_dupes_dir) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:  # Skip header
            cnn_clean_nodupes_texts.append(row[4])
        line_count += 1

CPU times: user 2min 44s, sys: 14.4 s, total: 2min 58s
Wall time: 3min 3s


In [28]:
len(cnn_clean_nodupes_texts)

6050

In [29]:
%%time
# CNN texts only
cnn_clean_nodupes_atall_texts = []
with open(cnn_no_dupesatall_dir) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:  # Skip header
            cnn_clean_nodupes_atall_texts.append(row[4])
        line_count += 1

CPU times: user 1min 46s, sys: 10 s, total: 1min 56s
Wall time: 1min 59s


In [53]:
%%time
# CNN Titles only
cnn_clean_nodupes_atall_titles = []
with open(cnn_no_dupesatall_dir) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:  # Skip header
            cnn_clean_nodupes_atall_titles.append(row[2])
        line_count += 1

CPU times: user 1min 55s, sys: 9.05 s, total: 2min 4s
Wall time: 2min 4s


In [30]:
len(cnn_clean_nodupes_atall_texts)

3942

In [31]:
%%time
# Smash all data together:
frames = [df_bbc_clean_nodupes_atall_csv, df_fox_clean_nodupes_atall_csv, df_rt_clean_nodupes_atall_csv]
df_master = pd.concat(frames, ignore_index=True)

CPU times: user 64.2 ms, sys: 200 ms, total: 264 ms
Wall time: 314 ms


In [32]:
df_master

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_link,rss_id,rss_published,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source,source_detail,pull_type,pull_date
0,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47063405,How has business been affected by Brexit so far?,[],With two months to go until the UK is due to l...,[],[''],set([]),"[u'Economics Correspondent', u'Dharshini David']",NaN,...,https://www.bbc.co.uk/news/business-47063405,https://www.bbc.co.uk/news/business-47063405,"Thu, 31 Jan 2019 00:03:21 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
1,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47066873,China's factory activity shrinks as slowdown w...,[],Chinese factory activity contracted for a seco...,[],[''],set([]),[],NaN,...,https://www.bbc.co.uk/news/business-47066873,https://www.bbc.co.uk/news/business-47066873,"Thu, 31 Jan 2019 02:53:39 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
2,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47057870,MPs say fast fashion brands inaction on ethics...,[],"Fashion retailers JD Sports, Sports Direct and...",[],[''],set([]),[],NaN,...,https://www.bbc.co.uk/news/business-47057870,https://www.bbc.co.uk/news/business-47057870,"Thu, 31 Jan 2019 00:12:15 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
3,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47055188,Brexit: Car investment halves as industry hits...,[],Investment in the UK car sector almost halved ...,[],[''],set([]),"[u'Business Reporter', u'Russell Hotten', u'Bb...",NaN,...,https://www.bbc.co.uk/news/business-47055188,https://www.bbc.co.uk/news/business-47055188,"Thu, 31 Jan 2019 00:04:41 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
4,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47065972,Facebook users continue to grow despite privac...,[],Facebook users have continued to rise despite ...,[],[''],set([]),[],NaN,...,https://www.bbc.co.uk/news/business-47065972,https://www.bbc.co.uk/news/business-47065972,"Wed, 30 Jan 2019 22:25:12 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
5,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47062145,Fed puts future rate rises on hold as pledges ...,[],The Federal Reserve has indicated it won't rai...,[],[''],set([]),[],NaN,...,https://www.bbc.co.uk/news/business-47062145,https://www.bbc.co.uk/news/business-47062145,"Wed, 30 Jan 2019 21:17:01 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
6,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47063403,Bosses' group head in abrupt departure,[],The boss of one of the UK's largest business l...,[],[''],set([]),[],NaN,...,https://www.bbc.co.uk/news/business-47063403,https://www.bbc.co.uk/news/business-47063403,"Wed, 30 Jan 2019 18:19:35 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
7,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47062142,Foxconn reconsiders Wisconsin factory plans,[],"Foxconn, which raised hopes of a US manufactur...",[],[''],set([]),[],NaN,...,https://www.bbc.co.uk/news/business-47062142,https://www.bbc.co.uk/news/business-47062142,"Wed, 30 Jan 2019 19:41:24 GMT","time.struct_time(tm_year=2019, tm_mon=1, tm_md...",NotFound,bbc_news_business,bbc,bbc_news_business,custom,2019-01-31
8,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47062146,Tesla reports profit as issues stabilise,[],Tesla made a profit of $139.5m (£10

### Topic Modelling
(based on https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/):

In [33]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [34]:
%%time
# Basic text cleaning:
df_master['clean_text'] = df_master['text'].apply(functions.clean_text_string,
                                                  keep_dbl_newline=False)
cnn_nodupes_atall_texts_cleaned = []
for text in cnn_clean_nodupes_atall_texts:
    cnn_nodupes_atall_texts_cleaned.append(functions.clean_text_string(text, keep_dbl_newline=False))

CPU times: user 10 s, sys: 105 ms, total: 10.1 s
Wall time: 10.6 s


In [35]:
# Enable stop words, for later clearing:
stop_words = stopwords.words('english')
# "bbc radio live" appears often, so add that to stop words; 'may' also seem turn up a lot, but not mean much:
stop_words.extend(['also', 'bbc', 'radio', 'live', 'may'])

In [36]:
# Convert our text strings to list:
data = df_master.clean_text.values.tolist()
data.extend(cnn_nodupes_atall_texts_cleaned)

In [37]:
# Extra text cleaning, just in case:

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [38]:
data_words = list(functions.sent_to_words(data))

#### Remove Stopwords, Make Bigrams/Trigrams and Lemmatize:

In [39]:
# Define functions for stopwords, bigrams, trigrams
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [40]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [41]:
%%time
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=70) # higher threshold fewer phrases.
#trigram = gensim.models.Phrases(bigram[data_words], threshold=70)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

CPU times: user 49.8 s, sys: 187 ms, total: 50 s
Wall time: 50.1 s


In [42]:
%%time
# Form Bigrams:
data_words_bigrams = make_bigrams(data_words_nostops)

CPU times: user 15.9 s, sys: 194 ms, total: 16.1 s
Wall time: 16.6 s


In [43]:
%%time
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

CPU times: user 26min 11s, sys: 1min 46s, total: 27min 57s
Wall time: 7min 23s


#### Create the Dictionary and Corpus needed for Topic Modeling:

In [44]:
%%time
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

CPU times: user 7.12 s, sys: 105 ms, total: 7.22 s
Wall time: 7.23 s


In [45]:
import time

#### Building LDA Model:

In [46]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        start = time.time()
        print('Topic modelling for num_topics=', num_topics)
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        #model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=num_topics, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        end = time.time()
        print('Finished num_topics', num_topics, 'with time', end - start)
        
    return model_list, coherence_values

In [9]:
%%time

# Can take a long time to run.
model_list2, coherence_values2 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=10, step=1)

NameError: name 'compute_coherence_values' is not defined

In [10]:
%%time
model_list3, coherence_values3 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=10, limit=15, step=1)

NameError: name 'compute_coherence_values' is not defined

In [11]:
%%time
model_list4, coherence_values4 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=15, limit=21, step=1)

NameError: name 'compute_coherence_values' is not defined

In [12]:
%%time
model_list5, coherence_values5 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=21, limit=25, step=1)

NameError: name 'compute_coherence_values' is not defined

In [13]:
%%time
model_list6, coherence_values6 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=25, limit=30, step=1)

NameError: name 'compute_coherence_values' is not defined

In [14]:
%%time
model_list7, coherence_values7 = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=30, limit=46, step=1)

NameError: name 'compute_coherence_values' is not defined

In [15]:
models = model_list2 + model_list3 + model_list4 + model_list5 + model_list6 + model_list7
coherence_values = coherence_values2 + coherence_values3 + coherence_values4 + coherence_values5 + coherence_values6 + coherence_values7

NameError: name 'model_list2' is not defined

In [ ]:
%%time
for model in models:
    name = 'models/2019_04_16_optimal_model_topics' + str(model.num_topics) + '.model'
    model.save(name)

In [ ]:
cohvals_path = 'models/2019_04_16_coherences_values.txt'

In [ ]:
%%time
# Load models:
models = []
directory_list = sorted(listdir('models/'))
for file in directory_list:
    if file.startswith('2019_04_16_optimal_model_topics') and file.endswith('.model'):
        model = gensim.models.LdaModel.load('models/' + file)
        models.append(model)

In [ ]:
%%time
# Recalculate coherence scores:
coherence_values = []
model_num = 0
for model in models:
    print('Model Num:', model_num)
    coherencemodel = CoherenceModel(model=model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())
    model_num += 1

In [ ]:
coherence_values

In [ ]:
%%time
with open(cohvals_path, 'wb') as fp:   #Pickling
    pickle.dump(coherence_values, fp)

In [ ]:
## Load Coherence values:
with open(cohvals_path, 'rb') as fp:
    coherence_values = pickle.load(fp)

In [ ]:
len(coherence_values)

In [ ]:
x = np.arange(2,46)

In [ ]:
# Create a trace
trace = go.Scatter(
    line = dict(width=1),
    x = x,
    y = coherence_values
)

layout = go.Layout(
    title=go.layout.Title(
        text='Topic Coherence',
        xref='paper',
        y=.88,
        font=dict(size=20)
    ),
     xaxis=dict(
         dtick=1,
         tick0=0,
         tickfont=dict(size=8),
         title='Number of Topics',
         titlefont=dict(size=14),
         tickangle=-45,
     ),
     yaxis=dict(
         dtick=.02,
         tick0=0,
         title='CV Coherence',
         tickfont=dict(size=10)
     ),
    annotations=[
    dict(
        x=23,
        y=0.5378038,
        xref='x',
        yref='y',
        text='(23, 0.5378038)',
        showarrow=True,
        arrowhead=3,
        ax=0,
        ay=-40
    )]
)


data = [trace]
fig1 = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

pio.write_image(fig1, 'papers/presentation/topic_coherence.pdf')

In [59]:
df_cnn = pd.DataFrame(columns=df_master.columns)

In [60]:
df_cnn['clean_text'] = cnn_nodupes_atall_texts_cleaned

In [65]:
df_cnn['title'] = cnn_clean_nodupes_atall_titles

In [61]:
df_cnn['source'] = 'cnn'

In [62]:
cnn_nodupes_atall_texts_cleaned[-1]

''

In [66]:
df_cnn

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,rss_id,rss_published,rss_published_parsed,rss_feedburner_origlink,paper_section_name,source,source_detail,pull_type,pull_date,clean_text
0,NaN,NaN,Retirement contribution limits will rise in 2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,Annual contribution limits to 401(k)s will inc...
1,NaN,NaN,Police have images of people they'd like to qu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,Investigators have located a surveillance came...
2,NaN,NaN,Ariana Grande's misspelled palm tattoo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,The singer got a palm tattoo with Japanese cha...
3,NaN,NaN,Maroon 5 letting Super Bowl show speak for itself,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,This year's halftime performers -- pop band Ma...
4,NaN,NaN,Hollywood supports Jussie Smollett after attac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,(CNN) Colleagues and admirers of Jussie Smolle...
5,NaN,NaN,Céline Dion on criticism she's too thin: 'Leav...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,Photos of the singer looking more svelte than ...
6,NaN,NaN,Gwyneth Paltrow's rep disputes ski-crash suit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,"Terry Sanderson, the skiier who claims he was ..."
7,NaN,NaN,"What's streaming on Netflix, Hulu, Amazon in F...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,"You just have to take it all ""One Day at a Tim..."
8,NaN,NaN,Nick Cannon stepping in for Wendy Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,"""The Wendy Williams Show"" announced Tuesday on..."
9,NaN,NaN,Ted Bundy: Netflix creeped out about serial ki...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,cnn,NaN,NaN,NaN,"Ted Bundy is back on a lot of people's minds, ..."


In [67]:
print(len(cnn_nodupes_atall_texts_cleaned))
print(len(df_master))

3942
11603


In [68]:
df_master_wcnn = pd.concat([df_master, df_cnn], ignore_index=True)

In [69]:
df_master_wcnn.to_csv('data/processed/2019_04_26_AllWCNN.csv', index=False)

### Adding dominant topics to data:

In [70]:
optimal_model = models[21] ## Model with 23 topics

In [71]:
optimal_model.get_topic_terms(1)

[(3185, 0.027598318),
 (2922, 0.024382347),
 (2394, 0.017087055),
 (2347, 0.016920447),
 (2582, 0.016741654),
 (4248, 0.015310848),
 (4509, 0.012955958),
 (1001, 0.012716234),
 (684, 0.011729562),
 (411, 0.010209484)]

In [74]:
%%time
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 18.8 µs


In [76]:
%%time
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

CPU times: user 41min 49s, sys: 4min 15s, total: 46min 5s
Wall time: 12min 36s


In [77]:
topic_modeling_df = pd.concat([df_master_wcnn, df_dominant_topic[['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords']]], axis=1)

In [80]:
topic_modeling_df_dropped = topic_modeling_df.drop(df_master_wcnn[df_master_wcnn.clean_text == ''].index)

In [82]:
topic_modeling_df_dropped.to_csv('data/processed/2019_04_26_AllWCNN_WTopics_noemptystr.csv', index=False)

#### Visualize Topics:

In [83]:
cnn_filter = (topic_modeling_df_dropped.source == 'cnn')
fox_filter = (topic_modeling_df_dropped.source == 'fox')
rt_filter = (topic_modeling_df_dropped.source == 'rt')
bbc_filter = (topic_modeling_df_dropped.source == 'bbc')

In [84]:
x = np.random.randn(500)
trace1 = go.Histogram(x=topic_modeling_df_dropped[cnn_filter]['Dominant_Topic'], opacity=0.75, name='CNN',
                      marker=dict(
                          line=dict(width=.8)))
trace2 = go.Histogram(x=topic_modeling_df_dropped[fox_filter]['Dominant_Topic'], opacity=0.75, name='Fox News',
                      marker=dict(
                          line=dict(width=.8)))
trace3 = go.Histogram(x=topic_modeling_df_dropped[rt_filter]['Dominant_Topic'], opacity=0.75, name='RT',
                      marker=dict(
                          line=dict(width=.8)))
trace4 = go.Histogram(x=topic_modeling_df_dropped[bbc_filter]['Dominant_Topic'], opacity=0.75, name='BBC',
                      marker=dict(
                          line=dict(width=.8)))

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    xaxis=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1
    ),
    title=go.layout.Title(
        text='Topic Distributions by Source',
        xref='paper',
        y=.88,
        font=dict(size=20)
    )
)

fig2 = tools.make_subplots(rows=2, cols=2)
fig2.append_trace(trace1, 1, 1)
fig2.append_trace(trace2, 1, 2)
fig2.append_trace(trace3, 2, 1)
fig2.append_trace(trace4, 2, 2)
fig2['layout'].update(title='Topic Distribution by Source',
                     xaxis1=dict(
                        tickmode='linear',
                        ticks='outside',
                        tick0=0,
                        dtick=1,
                         tickfont=dict(
            size=7
        )
                     ), 
                    yaxis1=dict(range=[0, 250], title='Count', 
                  dtick=25, tickfont=dict(
            size=7)),
         xaxis2=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,
                         tickfont=dict(
            size=7
        )
    ),yaxis2=dict(range=[0, 250], 
                  dtick=25, tickfont=dict(
            size=7)),
         xaxis3=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,title='Topic',
                         tickfont=dict(
            size=7
        )
    ),yaxis3=dict(range=[0, 250], title='Count', 
                  dtick=25, tickfont=dict(
            size=7)),
                     xaxis4=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,
         title='Topic',
                         tickfont=dict(
            size=7
        )
    ),yaxis4=dict(range=[0, 250], 
                  dtick=25, tickfont=dict(
            size=7)))

plotly.offline.iplot(fig2, filename='basic histogram')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [ ]:
pio.write_image(fig2, 'papers/presentation/topic_distribution.svg')

### Interpret Topics:

In [85]:
#%%time
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      19.469769        1       1  0.162466 -0.096936
2      11.600040        1       2  0.180008 -0.079828
5       7.548402        1       3  0.137362 -0.058050
17      5.695082        1       4  0.221770  0.126403
12      5.145532        1       5  0.124486 -0.034912
10      3.938830        1       6  0.134799  0.204824
13      3.819927        1       7  0.036737 -0.138865
9       3.818684        1       8 -0.041832 -0.038771
14      3.611831        1       9  0.010810  0.056223
16      3.539786        1      10  0.132541  0.014101
11      3.472906        1      11  0.018692 -0.191923
4       3.353090        1      12 -0.131907  0.081438
15      3.230227        1      13 -0.185969 -0.110960
20      3.193549        1      14 -0.032575 -0.142156
1       3.165687        1      15 -0.026028  0.091004
8       2.588670        1      16  0.181801  0.081790
22      2.287472        1      17 -0.111019 -0.108508
19      2.260243        1      18 -0.168581  0.127602
18      2.047637        1      19 -0.132702  0.060687
7       1.715212        1      20 -0.123820 -0.166439
21      1.609229        1      21 -0.156708  0.037003
6       1.477905        1      22 -0.209975  0.142933
3       1.410291        1      23 -0.020356  0.143341, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
1041   Default  21366.000000        trump  21366.000000  30.0000  30.0000
271    Default  67789.000000          say  67789.000000  29.0000  29.0000
210    Default  22186.000000          not  22186.000000  28.0000  28.0000
1484   Default  10671.000000    president  10671.000000  27.0000  27.0000
229    Default  15588.000000       people  15588.000000  26.0000  26.0000
1738   Default   6931.000000        woman   6931.000000  25.0000  25.0000
15190  Default  10832.000000          cnn  10832.000000  24.0000  24.0000
53     Default   8036.000000      company   8036.000000  23.0000  23.0000
84     Default  13231.000000           do  13231.000000  22.0000  22.0000
140    Default  12901.000000           go  12901.000000  21.0000  21.0000
737    Default  18156.000000        would  18156.000000  20.0000  20.0000
270    Default  12343.000000            s  12343.000000  19.0000  19.0000
1174   Default   7140.000000       family   7140.000000  18.0000  18.0000
26     Default  11421.000000           be  11421.000000  17.0000  17.0000
1067   Default  12075.000000          get  12075.000000  16.0000  16.0000
330    Default  17646.000000         year  17646.000000  15.0000  15.0000
71     Default   5560.000000         deal   5560.000000  14.0000  14.0000
2403   Default   5663.000000        child   5663.000000  13.0000  13.0000
588    Default   9385.000000          use   9385.000000  12.0000  12.0000
1387   Default   4332.000000       police   4332.000000  11.0000  11.0000
623    Default   7720.000000      country   7720.000000  10.0000  10.0000
10289  Default   3234.000000      federal   3234.000000   9.0000   9.0000
3185   Default   5664.000000         team   5664.000000   8.0000   8.0000
1821   Default   4080.000000     democrat   4080.000000   7.0000   7.0000
733    Default   4520.000000         vote   4520.000000   6.0000   6.0000
1054   Default   6006.000000         case   6006.000000   5.0000   5.0000
259    Default   8295.000000       report   8295.000000   4.0000   4.0000
472    Default   6091.000000   government   6091.000000   3.0000   3.0000
35     Default   4245.000000     business   4245.000000   2.0000   2.0000
47     Default   4095.000000         city   4095.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
4040   Topic23    155.537415       priest    156.365845   4.2561  -5.8363
33014  Topic23    150.603699       tapper    151.432129   4.2559  -5.8686
5847   Topic23   

In [86]:
topic_filter = (topic_modeling_df_dropped['Dominant_Topic'] == 17)
lookattopic = topic_modeling_df_dropped[['title','Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'clean_text']][topic_filter].sort_values(by=['Topic_Perc_Contrib'], ascending=False)
lookattopic.head(100)

,title,Dominant_Topic,Topic_Perc_Contrib,Keywords,clean_text
7854,Trump backs big ‘protests against ex-dictator ...,17.0,0.6455,"trump, president, congress, white_house, admin...",Venezuela’s opposition leader and self-declare...
8162,Cuba slams US for ‘meddling’ accusation on Ven...,17.0,0.6416,"trump, president, congress, white_house, admin...",Cuban Foreign Minister Bruno Rodriguez has rej...
7932,US threatens ‘serious consequences’ if Venezue...,17.0,0.6329,"trump, president, congress, white_house, admin...",National Security Advisor John Bolton has thre...
12990,Trump ramps up pressure on Venezuela's Maduro ...,17.0,0.6078,"trump, president, congress, white_house, admin...",Washington (CNN) President Donald Trump on Mon...
8126,Maduro slams Trump in English (VIDEO) — RT Wor...,17.0,0.6041,"trump, president, congress, white_house, admin...",Venezuelan leader Nicolas Maduro addressed Don...
7851,US sanctions Nicaragua oil company over Venezu...,17.0,0.6018,"trump, president, congress, white_house, admin...","Washington has imposed sanctions on Albanisa, ..."
3269,Venezuela crisis: Juan Guaidó to return after ...,17.0,0.6002,"trump, president, congress, white_house, admin...",Venezuela's self-proclaimed interim president ...
398,Venezuela crisis: US sending aid 'at Guaidó's ...,17.0,0.5915,"trump, president, congress, white_house, admin...",The US says it is sending aid to crisis-hit Ve...
8951,US is openly pushing Venezuela’s army into a coup,17.0,0.5733,"trump, president, congress, white_house, admin...",Forced regime change has apparently become Was...
106,"Venezuela opposition 'has met military', says ...",17.0,0.5715,"trump, president, congress, white_house, admin...",Venezuela's opposition leader Juan Guaidó has ...


In [88]:
peak_idx = 7854
print(lookattopic.Keywords[peak_idx])
print(lookattopic.clean_text[peak_idx])

trump, president, congress, white_house, administration, mueller, country, special_counsel, government, russia
Venezuela’s opposition leader and self-declared ‘interim president’ Juan Guaido chatted by phone with US President Donald Trump, with Trump giving the thumbs up to “large protests” against “former dictator Maduro.” The phone call occurred Wednesday as anti-government demonstrators in Venezuela were about to take to the streets for the second time in a week. Trump “noted the importance” of the protests against “former dictator Maduro,” and vowed “strong support for Venezuela’s fight to regain its democracy,” according to White House Press Secretary Sarah Sanders. Nicolas Maduro remains Venezuela’s elected leader, despite a number of countries declaring their support for Guaidó’s claim as the acting head. Trump, meanwhile, congratulated Guaidó on his “historic assumption of the presidency.” Neither President Maduro or predecessor Hugo Chavez ever received a phone call from a US 

In [1]:
topic_translation = {0: 'Unknown',
                     1: 'Sports',
                     2: 'Political Investigation (Cohen/Virginia)',
                     3: 'Military Nuclear Technology',
                     4: 'Film Industry',
                     5: 'Cybersecurity',
                     6: 'Entertainment Industry',
                     7: 'Health',
                     8: 'Brexit',
                     9: 'Space Industry',
                     10: 'US Campaign Politics',
                     11: 'Medical Research',
                     12: 'Economy',
                     13: 'Emergencies',
                     14: 'Religious Intolerance',
                     15: 'Family Affairs',
                     16: 'International Affairs (Middle East and South Asia)',
                     17: 'International Affairs (Latin America)',
                     18: 'Corporate News',
                     19: 'Fashion/Issues of Race',
                     20: 'Crime',
                     21: 'Social Media',
                     22: 'Nature and Environment'}

In [ ]:
%%time

# Save Topic Dictionary:
topic_dict_path = 'models/2019_04_16_Topic_Dictionary(23_topics)'
with open(topic_dict_path, 'wb') as fp:   #Pickling
    pickle.dump(topic_translation, fp)

In [90]:
%%time
topic_modeling_df_dropped['Topic'] = topic_modeling_df_dropped['Dominant_Topic'].map(topic_translation)

CPU times: user 8.9 ms, sys: 4.22 ms, total: 13.1 ms
Wall time: 19.6 ms


In [91]:
# Save results:
topic_modeling_df_dropped.to_csv('data/processed/2019_04_26_AllWCNN_WTopics_noemptystr_inttopics.csv', index=False)

In [ ]:
# Output the topics-probability table:
model_topics = optimal_model.show_topics(formatted=False, num_words=20)
#pprint(optimal_model.print_topics(num_words=20))
model_topics.sort()
with open('papers/presentation/topics_probs.csv', mode='w') as csv_file:
    fieldnames = ['Topic', 'Token', 'Weight']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for topic_num, words in model_topics:
        topic = topic_translation[topic_num]
        for word, prop in words:
            writer.writerow({'Topic': topic, 'Token': word, 'Weight': prop})

### NER:

In [8]:
%%time
import spacy
nlp_ner = spacy.load("en_core_web_lg")

CPU times: user 8.94 s, sys: 2.15 s, total: 11.1 s
Wall time: 13.1 s


In [144]:
%%time
i = 0
ner_tokens = []
ner_entities = []
for a_title in topic_modeling_df_dropped.title:
    print('WORKING ON TITLE', i)
    doc = nlp_ner(a_title)
    ner_tokens.append(doc.ents)
    ner_tup = []
    for ent in doc.ents:
        ner_tup.append(ent.label_)
    ner_entities.append(ner_tup)
    i += 1

WORKING ON TITLE 0
WORKING ON TITLE 1
WORKING ON TITLE 2
WORKING ON TITLE 3
WORKING ON TITLE 4
WORKING ON TITLE 5
WORKING ON TITLE 6
WORKING ON TITLE 7
WORKING ON TITLE 8
WORKING ON TITLE 9
WORKING ON TITLE 10
WORKING ON TITLE 11
WORKING ON TITLE 12
WORKING ON TITLE 13
WORKING ON TITLE 14
WORKING ON TITLE 15
WORKING ON TITLE 16
WORKING ON TITLE 17
WORKING ON TITLE 18
WORKING ON TITLE 19
WORKING ON TITLE 20
WORKING ON TITLE 21
WORKING ON TITLE 22
WORKING ON TITLE 23
WORKING ON TITLE 24
WORKING ON TITLE 25
WORKING ON TITLE 26
WORKING ON TITLE 27
WORKING ON TITLE 28
WORKING ON TITLE 29
WORKING ON TITLE 30
WORKING ON TITLE 31
WORKING ON TITLE 32
WORKING ON TITLE 33
WORKING ON TITLE 34
WORKING ON TITLE 35
WORKING ON TITLE 36
WORKING ON TITLE 37
WORKING ON TITLE 38
WORKING ON TITLE 39
WORKING ON TITLE 40
WORKING ON TITLE 41
WORKING ON TITLE 42
WORKING ON TITLE 43
WORKING ON TITLE 44
WORKING ON TITLE 45
WORKING ON TITLE 46
WORKING ON TITLE 47
WORKING ON TITLE 48
WORKING ON TITLE 49
WORKING ON

WORKING ON TITLE 399
WORKING ON TITLE 400
WORKING ON TITLE 401
WORKING ON TITLE 402
WORKING ON TITLE 403
WORKING ON TITLE 404
WORKING ON TITLE 405
WORKING ON TITLE 406
WORKING ON TITLE 407
WORKING ON TITLE 408
WORKING ON TITLE 409
WORKING ON TITLE 410
WORKING ON TITLE 411
WORKING ON TITLE 412
WORKING ON TITLE 413
WORKING ON TITLE 414
WORKING ON TITLE 415
WORKING ON TITLE 416
WORKING ON TITLE 417
WORKING ON TITLE 418
WORKING ON TITLE 419
WORKING ON TITLE 420
WORKING ON TITLE 421
WORKING ON TITLE 422
WORKING ON TITLE 423
WORKING ON TITLE 424
WORKING ON TITLE 425
WORKING ON TITLE 426
WORKING ON TITLE 427
WORKING ON TITLE 428
WORKING ON TITLE 429
WORKING ON TITLE 430
WORKING ON TITLE 431
WORKING ON TITLE 432
WORKING ON TITLE 433
WORKING ON TITLE 434
WORKING ON TITLE 435
WORKING ON TITLE 436
WORKING ON TITLE 437
WORKING ON TITLE 438
WORKING ON TITLE 439
WORKING ON TITLE 440
WORKING ON TITLE 441
WORKING ON TITLE 442
WORKING ON TITLE 443
WORKING ON TITLE 444
WORKING ON TITLE 445
WORKING ON TI

WORKING ON TITLE 805
WORKING ON TITLE 806
WORKING ON TITLE 807
WORKING ON TITLE 808
WORKING ON TITLE 809
WORKING ON TITLE 810
WORKING ON TITLE 811
WORKING ON TITLE 812
WORKING ON TITLE 813
WORKING ON TITLE 814
WORKING ON TITLE 815
WORKING ON TITLE 816
WORKING ON TITLE 817
WORKING ON TITLE 818
WORKING ON TITLE 819
WORKING ON TITLE 820
WORKING ON TITLE 821
WORKING ON TITLE 822
WORKING ON TITLE 823
WORKING ON TITLE 824
WORKING ON TITLE 825
WORKING ON TITLE 826
WORKING ON TITLE 827
WORKING ON TITLE 828
WORKING ON TITLE 829
WORKING ON TITLE 830
WORKING ON TITLE 831
WORKING ON TITLE 832
WORKING ON TITLE 833
WORKING ON TITLE 834
WORKING ON TITLE 835
WORKING ON TITLE 836
WORKING ON TITLE 837
WORKING ON TITLE 838
WORKING ON TITLE 839
WORKING ON TITLE 840
WORKING ON TITLE 841
WORKING ON TITLE 842
WORKING ON TITLE 843
WORKING ON TITLE 844
WORKING ON TITLE 845
WORKING ON TITLE 846
WORKING ON TITLE 847
WORKING ON TITLE 848
WORKING ON TITLE 849
WORKING ON TITLE 850
WORKING ON TITLE 851
WORKING ON TI

WORKING ON TITLE 1201
WORKING ON TITLE 1202
WORKING ON TITLE 1203
WORKING ON TITLE 1204
WORKING ON TITLE 1205
WORKING ON TITLE 1206
WORKING ON TITLE 1207
WORKING ON TITLE 1208
WORKING ON TITLE 1209
WORKING ON TITLE 1210
WORKING ON TITLE 1211
WORKING ON TITLE 1212
WORKING ON TITLE 1213
WORKING ON TITLE 1214
WORKING ON TITLE 1215
WORKING ON TITLE 1216
WORKING ON TITLE 1217
WORKING ON TITLE 1218
WORKING ON TITLE 1219
WORKING ON TITLE 1220
WORKING ON TITLE 1221
WORKING ON TITLE 1222
WORKING ON TITLE 1223
WORKING ON TITLE 1224
WORKING ON TITLE 1225
WORKING ON TITLE 1226
WORKING ON TITLE 1227
WORKING ON TITLE 1228
WORKING ON TITLE 1229
WORKING ON TITLE 1230
WORKING ON TITLE 1231
WORKING ON TITLE 1232
WORKING ON TITLE 1233
WORKING ON TITLE 1234
WORKING ON TITLE 1235
WORKING ON TITLE 1236
WORKING ON TITLE 1237
WORKING ON TITLE 1238
WORKING ON TITLE 1239
WORKING ON TITLE 1240
WORKING ON TITLE 1241
WORKING ON TITLE 1242
WORKING ON TITLE 1243
WORKING ON TITLE 1244
WORKING ON TITLE 1245
WORKING ON

WORKING ON TITLE 1574
WORKING ON TITLE 1575
WORKING ON TITLE 1576
WORKING ON TITLE 1577
WORKING ON TITLE 1578
WORKING ON TITLE 1579
WORKING ON TITLE 1580
WORKING ON TITLE 1581
WORKING ON TITLE 1582
WORKING ON TITLE 1583
WORKING ON TITLE 1584
WORKING ON TITLE 1585
WORKING ON TITLE 1586
WORKING ON TITLE 1587
WORKING ON TITLE 1588
WORKING ON TITLE 1589
WORKING ON TITLE 1590
WORKING ON TITLE 1591
WORKING ON TITLE 1592
WORKING ON TITLE 1593
WORKING ON TITLE 1594
WORKING ON TITLE 1595
WORKING ON TITLE 1596
WORKING ON TITLE 1597
WORKING ON TITLE 1598
WORKING ON TITLE 1599
WORKING ON TITLE 1600
WORKING ON TITLE 1601
WORKING ON TITLE 1602
WORKING ON TITLE 1603
WORKING ON TITLE 1604
WORKING ON TITLE 1605
WORKING ON TITLE 1606
WORKING ON TITLE 1607
WORKING ON TITLE 1608
WORKING ON TITLE 1609
WORKING ON TITLE 1610
WORKING ON TITLE 1611
WORKING ON TITLE 1612
WORKING ON TITLE 1613
WORKING ON TITLE 1614
WORKING ON TITLE 1615
WORKING ON TITLE 1616
WORKING ON TITLE 1617
WORKING ON TITLE 1618
WORKING ON

...

WORKING ON TITLE 13913
WORKING ON TITLE 13914
WORKING ON TITLE 13915
WORKING ON TITLE 13916
WORKING ON TITLE 13917
WORKING ON TITLE 13918
WORKING ON TITLE 13919
WORKING ON TITLE 13920
WORKING ON TITLE 13921
WORKING ON TITLE 13922
WORKING ON TITLE 13923
WORKING ON TITLE 13924
WORKING ON TITLE 13925
WORKING ON TITLE 13926
WORKING ON TITLE 13927
WORKING ON TITLE 13928
WORKING ON TITLE 13929
WORKING ON TITLE 13930
WORKING ON TITLE 13931
WORKING ON TITLE 13932
WORKING ON TITLE 13933
WORKING ON TITLE 13934
WORKING ON TITLE 13935
WORKING ON TITLE 13936
WORKING ON TITLE 13937
WORKING ON TITLE 13938
WORKING ON TITLE 13939
WORKING ON TITLE 13940
WORKING ON TITLE 13941
WORKING ON TITLE 13942
WORKING ON TITLE 13943
WORKING ON TITLE 13944
WORKING ON TITLE 13945
WORKING ON TITLE 13946
WORKING ON TITLE 13947
WORKING ON TITLE 13948
WORKING ON TITLE 13949
WORKING ON TITLE 13950
WORKING ON TITLE 13951
WORKING ON TITLE 13952
WORKING ON TITLE 13953
WORKING ON TITLE 13954
WORKING ON TITLE 13955
WORKING ON 

WORKING ON TITLE 14281
WORKING ON TITLE 14282
WORKING ON TITLE 14283
WORKING ON TITLE 14284
WORKING ON TITLE 14285
WORKING ON TITLE 14286
WORKING ON TITLE 14287
WORKING ON TITLE 14288
WORKING ON TITLE 14289
WORKING ON TITLE 14290
WORKING ON TITLE 14291
WORKING ON TITLE 14292
WORKING ON TITLE 14293
WORKING ON TITLE 14294
WORKING ON TITLE 14295
WORKING ON TITLE 14296
WORKING ON TITLE 14297
WORKING ON TITLE 14298
WORKING ON TITLE 14299
WORKING ON TITLE 14300
WORKING ON TITLE 14301
WORKING ON TITLE 14302
WORKING ON TITLE 14303
WORKING ON TITLE 14304
WORKING ON TITLE 14305
WORKING ON TITLE 14306
WORKING ON TITLE 14307
WORKING ON TITLE 14308
WORKING ON TITLE 14309
WORKING ON TITLE 14310
WORKING ON TITLE 14311
WORKING ON TITLE 14312
WORKING ON TITLE 14313
WORKING ON TITLE 14314
WORKING ON TITLE 14315
WORKING ON TITLE 14316
WORKING ON TITLE 14317
WORKING ON TITLE 14318
WORKING ON TITLE 14319
WORKING ON TITLE 14320
WORKING ON TITLE 14321
WORKING ON TITLE 14322
WORKING ON TITLE 14323
WORKING ON 

WORKING ON TITLE 14639
WORKING ON TITLE 14640
WORKING ON TITLE 14641
WORKING ON TITLE 14642
WORKING ON TITLE 14643
WORKING ON TITLE 14644
WORKING ON TITLE 14645
WORKING ON TITLE 14646
WORKING ON TITLE 14647
WORKING ON TITLE 14648
WORKING ON TITLE 14649
WORKING ON TITLE 14650
WORKING ON TITLE 14651
WORKING ON TITLE 14652
WORKING ON TITLE 14653
WORKING ON TITLE 14654
WORKING ON TITLE 14655
WORKING ON TITLE 14656
WORKING ON TITLE 14657
WORKING ON TITLE 14658
WORKING ON TITLE 14659
WORKING ON TITLE 14660
WORKING ON TITLE 14661
WORKING ON TITLE 14662
WORKING ON TITLE 14663
WORKING ON TITLE 14664
WORKING ON TITLE 14665
WORKING ON TITLE 14666
WORKING ON TITLE 14667
WORKING ON TITLE 14668
WORKING ON TITLE 14669
WORKING ON TITLE 14670
WORKING ON TITLE 14671
WORKING ON TITLE 14672
WORKING ON TITLE 14673
WORKING ON TITLE 14674
WORKING ON TITLE 14675
WORKING ON TITLE 14676
WORKING ON TITLE 14677
WORKING ON TITLE 14678
WORKING ON TITLE 14679
WORKING ON TITLE 14680
WORKING ON TITLE 14681
WORKING ON 

WORKING ON TITLE 15010
WORKING ON TITLE 15011
WORKING ON TITLE 15012
WORKING ON TITLE 15013
WORKING ON TITLE 15014
WORKING ON TITLE 15015
WORKING ON TITLE 15016
WORKING ON TITLE 15017
WORKING ON TITLE 15018
WORKING ON TITLE 15019
WORKING ON TITLE 15020
WORKING ON TITLE 15021
WORKING ON TITLE 15022
WORKING ON TITLE 15023
WORKING ON TITLE 15024
WORKING ON TITLE 15025
WORKING ON TITLE 15026
WORKING ON TITLE 15027
WORKING ON TITLE 15028
WORKING ON TITLE 15029
WORKING ON TITLE 15030
WORKING ON TITLE 15031
WORKING ON TITLE 15032
WORKING ON TITLE 15033
WORKING ON TITLE 15034
WORKING ON TITLE 15035
WORKING ON TITLE 15036
WORKING ON TITLE 15037
WORKING ON TITLE 15038
WORKING ON TITLE 15039
WORKING ON TITLE 15040
WORKING ON TITLE 15041
WORKING ON TITLE 15042
WORKING ON TITLE 15043
WORKING ON TITLE 15044
WORKING ON TITLE 15045
WORKING ON TITLE 15046
WORKING ON TITLE 15047
WORKING ON TITLE 15048
WORKING ON TITLE 15049
WORKING ON TITLE 15050
WORKING ON TITLE 15051
WORKING ON TITLE 15052
WORKING ON 

WORKING ON TITLE 15381
WORKING ON TITLE 15382
WORKING ON TITLE 15383
WORKING ON TITLE 15384
WORKING ON TITLE 15385
WORKING ON TITLE 15386
WORKING ON TITLE 15387
WORKING ON TITLE 15388
WORKING ON TITLE 15389
WORKING ON TITLE 15390
WORKING ON TITLE 15391
WORKING ON TITLE 15392
WORKING ON TITLE 15393
WORKING ON TITLE 15394
WORKING ON TITLE 15395
WORKING ON TITLE 15396
WORKING ON TITLE 15397
WORKING ON TITLE 15398
WORKING ON TITLE 15399
WORKING ON TITLE 15400
WORKING ON TITLE 15401
WORKING ON TITLE 15402
WORKING ON TITLE 15403
WORKING ON TITLE 15404
WORKING ON TITLE 15405
WORKING ON TITLE 15406
WORKING ON TITLE 15407
WORKING ON TITLE 15408
WORKING ON TITLE 15409
WORKING ON TITLE 15410
WORKING ON TITLE 15411
WORKING ON TITLE 15412
WORKING ON TITLE 15413
WORKING ON TITLE 15414
WORKING ON TITLE 15415
WORKING ON TITLE 15416
WORKING ON TITLE 15417
WORKING ON TITLE 15418
WORKING ON TITLE 15419
WORKING ON TITLE 15420
WORKING ON TITLE 15421
WORKING ON TITLE 15422
WORKING ON TITLE 15423
WORKING ON 

In [147]:
print(len(ner_tokens))
print(len(ner_entities))

15500
15500


In [149]:
topic_modeling_df_dropped['ner_tokens'] = ner_tokens
topic_modeling_df_dropped['ner_entities'] = ner_entities

In [155]:
topic_modeling_df_dropped[['title','ner_tokens', 'ner_entities']]

,title,ner_tokens,ner_entities
0,How has business been affected by Brexit so far?,"((Brexit),)",[ORG]
1,China's factory activity shrinks as slowdown w...,"((China),)",[GPE]
2,MPs say fast fashion brands inaction on ethics...,(),[]
3,Brexit: Car investment halves as industry hits...,"((Brexit),)",[GPE]
4,Facebook users continue to grow despite privac...,"((Facebook),)",[ORG]
5,Fed puts future rate rises on hold as pledges ...,"((Fed),)",[ORG]
6,Bosses' group head in abrupt departure,(),[]
7,Foxconn reconsiders Wisconsin factory plans,"((Foxconn), (Wisconsin))","[ORG, GPE]"
8,Tesla reports profit as issues stabilise,"((Tesla),)",[ORG]
9,Barclays shifts billions of pounds to Dublin b...,"((Barclays), (billions, of, pounds), (Dublin),...","[ORG, MONEY, GPE, GPE]"


In [164]:
# Save results:
topic_modeling_df_dropped.to_csv('data/processed/2019_04_26_AllWCNN_WTopics_noemptystr_inttopics_NER.csv', index=False)

In [3]:
# Load back:
topic_modeling_df_dropped = pd.read_csv('data/processed/2019_04_26_AllWCNN_WTopics_noemptystr_inttopics_NER.csv', index_col=None)

In [4]:
topic_modeling_df_dropped

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords,Topic,ner_tokens,ner_entities
0,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47063405,How has business been affected by Brexit so far?,[],With two months to go until the UK is due to l...,[],[''],set([]),"[u'Economics Correspondent', u'Dharshini David']",NaN,...,bbc_news_business,custom,2019-01-31,With two months to go until the UK is due to l...,12.0,0.3840,"year, company, say, last, new, cost, business,...",Economy,"(Brexit,)",['ORG']
1,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47066873,China's factory activity shrinks as slowdown w...,[],Chinese factory activity contracted for a seco...,[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,Chinese factory activity contracted for a seco...,12.0,0.5390,"year, company, say, last, new, cost, business,...",Economy,"(China,)",['GPE']
2,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47057870,MPs say fast fashion brands inaction on ethics...,[],"Fashion retailers JD Sports, Sports Direct and...",[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,"Fashion retailers JD Sports, Sports Direct and...",12.0,0.2574,"year, company, say, last, new, cost, business,...",Economy,(),[]
3,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47055188,Brexit: Car investment halves as industry hits...,[],Investment in the UK car sector almost halved ...,[],[''],set([]),"[u'Business Reporter', u'Russell Hotten', u'Bb...",NaN,...,bbc_news_business,custom,2019-01-31,Investment in the UK car sector almost halved ...,12.0,0.5330,"year, company, say, last, new, cost, business,...",Economy,"(Brexit,)",['GPE']
4,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47065972,Facebook users continue to grow despite privac...,[],Facebook users have continued to rise despite ...,[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,Facebook users have continued to rise despite ...,12.0,0.2705,"year, company, say, last, new, cost, business,...",Economy,"(Facebook,)",['ORG']
5,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47062145,Fed puts future rate rises on hold as pledges ...,[],The Federal Reserve has indicated it won't rai...,[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,The Federal Reserve has indicated it won't rai...,12.0,0.3369,"year, company, say, last, new, cost, business,...",Economy,"(Fed,)",['ORG']
6,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47063403,Bosses' group head in abrupt departure,[],The boss of one of the UK's largest business l...,[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,The boss of one of the UK's largest business l...,2.0,0.4538,"say, cnn, report, tell, investigation, stateme...",Political Investigation (Cohen/Virginia),(),[]
7,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47062142,Foxconn reconsiders Wisconsin factory plans,[],"Foxconn, which raised hopes of a US manufactur...",[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,"Foxconn, which raised hopes of a US manufactur...",12.0,0.4733,"year, company, say, last, new, cost, business,...",Economy,"(Foxconn, Wisconsin)","['ORG', 'GPE']"
8,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47062146,Tesla reports profit as issues stabilise,[],Tesla made a profit of $139.5m (£106.4m) in th...,[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,Tesla made a profit of $139.5m (£106.4m) in th...,12.0,0.5972,"year, company, say, last, new, cost, business,...",Economy,"(Tesla,)",['ORG']
9,https://www.bbc.com/news,https://www.bbc.co.uk/news/business-47060676,Barclays shifts billions of pounds to Dublin b...,[],Barclays is moving €190bn (£166bn) of assets t...,[],[''],set([]),[],NaN,...,bbc_news_business,custom,2019-01-31,Barclays is moving €190bn (£166bn) of 

In [5]:
# All NER entities from Spacy:
all_ner = ['PERSON',
    'NORP',
    'FAC',
    'ORG',
    'GPE',
    'LOC',
    'PRODUCT',
    'EVENT',
    'WORK_OF_ART',
    'LAW',
    'LANGUAGE',
    'DATE',
    'TIME',
    'PERCENT',
    'MONEY',
    'QUANTITY',
    'ORDINAL',
    'CARDINAL']

Alternative way of NER:

In [6]:
topic_modeling_df_dropped.title

0         How has business been affected by Brexit so far?
1        China's factory activity shrinks as slowdown w...
2        MPs say fast fashion brands inaction on ethics...
3        Brexit: Car investment halves as industry hits...
4        Facebook users continue to grow despite privac...
5        Fed puts future rate rises on hold as pledges ...
6                   Bosses' group head in abrupt departure
7              Foxconn reconsiders Wisconsin factory plans
8                 Tesla reports profit as issues stabilise
9        Barclays shifts billions of pounds to Dublin b...
10       Electric checks in rented homes confirmed by g...
11       Brexit: Firms may speed up no-deal plans, says...
12       Carlos Ghosn says 'plot and treason' behind ar...
13       Brazil miner Vale cuts output as disaster deat...
14                 Credit card use squeezed over Christmas
15         DJI pledges painful action to tackle corruption
16                Tonnes of chicken nuggets recalled in 

In [27]:
%%time
i = 0
ner_tokens = {}
ner_entities = []
ner_df = pd.DataFrame(columns=['article_idx', 'ner_token', 'ner_label'], index=topic_modeling_df_dropped.index)
for a_title in topic_modeling_df_dropped.title:
    print('WORKING ON TITLE', i)
    doc = nlp_ner(a_title)
    for ent in doc.ents:
        a_row = {'article_idx': i, 
                 'ner_token': ent, 
                 'ner_label': ent.label_}
        break
        ner_df = ner_df.append(a_row, ignore_index=True)
    break
    i += 1

WORKING ON TITLE 0
CPU times: user 49.9 ms, sys: 7 ms, total: 56.9 ms
Wall time: 22.7 ms


In [28]:
a_row

{'article_idx': 0, 'ner_label': 'ORG', 'ner_token': Brexit}

Save, just in case:

In [11]:
ner_df.to_csv('data/processed/2019_04_30_NER_df.csv', index=False)

In [30]:
ner_df = pd.read_csv('data/processed/2019_04_30_NER_df.csv')

In [188]:
ner_df_dropna = ner_df.dropna(axis=0, how='all', inplace=False)

In [32]:
counts_ner = ner_df.groupby('ner_token').count()

In [181]:
counts_ner

,article_idx,ner_label
ner_token,,
#,8,8
$1 billion,1,1
$1 million,3,3
$1 million dollars,1,1
$1 trillion,2,2
"$1,000+",1,1
$1.2 billion,1,1
$1.2 million,1,1
$1.2m,1,1


In [167]:
print(counts_ner[counts_ner.ner_label > 60].sort_values(by=['ner_label'], ascending=False))

                  article_idx  ner_label
ner_token                               
RT World News            1104       1104
US                        909        909
Trump                     828        828
RT USA News               549        549
RT Newsline               464        464
RT Sport News             449        449
UK                        337        337
Brexit                    316        316
Russia                    289        289
Venezuela                 252        252
Russian                   246        246
RT Business News          241        241
China                     199        199
India                     173        173
Facebook                  169        169
2020                      168        168
EU                        151        151
first                     151        151
Pakistan                  137        137
California                137        137
Mueller                   128        128
May                       120        120
Democrats       

In [141]:
top_ner_df = counts_ner[counts_ner.ner_label > 60].sort_values(by=['ner_label'], ascending=False)

In [143]:
top_ner_df.to_csv('data/processed/2019_05_01_ner_counts_plus60.csv')

In [144]:
top_ner_df_csv = pd.read_csv('data/processed/2019_05_01_ner_counts_plus60.csv')

In [147]:
counts_ner

,article_idx,ner_label
ner_token,,
#,8,8
$1 billion,1,1
$1 million,3,3
$1 million dollars,1,1
$1 trillion,2,2
"$1,000+",1,1
$1.2 billion,1,1
$1.2 million,1,1
$1.2m,1,1


In [206]:
# Clean-up 'virginia'
virginia_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('virginia')]
virginia_ner = virginia_ner[~virginia_ner.std_ner_token.str.contains('west virginia')]

In [223]:
# Clean-up 'cohen':
cohen_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('cohen')]
# Delete distractions:
cohen_ner = cohen_ner[~cohen_ner.std_ner_token.str.contains('andy')]
cohen_ner = cohen_ner[~cohen_ner.std_ner_token.str.contains('richard')]

In [231]:
# Clean-up 'mueller': Looks good already
mueller_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('mueller')]
mueller_ner.std_ner_token.unique()

array(['mueller', 'robert mueller', "bob mueller'", "robert mueller's",
       'mueller report', 'mueller trump', 'barr-mueller', 'new mueller'],
      dtype=object)

In [246]:
# Clean-up 'jussie smollet': 
smollet_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('jussie')]
smollet_ner_2 = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('smollett')]
smollet_ner_3 = pd.concat([smollet_ner, smollet_ner_2]).drop_duplicates()

In [336]:
# Clean-up 'Christchurch': 
christchurch_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('christchurch')]
christchurch_ner.count()

article_idx      63
ner_token        63
ner_label        63
std_ner_token    63
dtype: int64

In [254]:
# Clean-up 'iran': 
iran_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('iran')]
print(iran_ner.std_ner_token.unique())
iran_ner = iran_ner[~iran_ner.std_ner_token.str.contains('miranda')]
iran_ner.head()

['iran' 'iranian' 'iranian islamic revolution' "miranda lambert's"
 'russia & iran' 'iran & turkey' 'iranians' 'turkish & iranian'
 'miranda lambert']


,article_idx,ner_token,ner_label,std_ner_token
15974,346.0,Iran,GPE,iran
16279,509.0,Iran,GPE,iran
17230,1114.0,Iran,GPE,iran
17780,1437.0,Iranian,NORP,iranian
18389,1784.0,Iran,GPE,iran


In [324]:
# Clean-up 'kim': 
kim_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('kim')]
exclude_kim_1 = ~ner_df_dropna.std_ner_token.str.contains('kimmel')
exclude_kim_2 = ~ner_df_dropna.std_ner_token.str.contains('bok-dong')
exclude_kim_3 = ~ner_df_dropna.std_ner_token.str.contains('kimbolton')
exclude_kim_4 = ~ner_df_dropna.std_ner_token.str.contains('davis')
exclude_kim_5 = ~ner_df_dropna.std_ner_token.str.contains('kardashian')
exclude_kim_6 = ~ner_df_dropna.std_ner_token.str.contains('kimmich')
exclude_kim_7 = ~ner_df_dropna.std_ner_token.str.contains('kimpembe')
exclude_kim_8 = ~ner_df_dropna.std_ner_token.str.contains('cattrall')
exclude_kim_9 = ~ner_df_dropna.std_ner_token.str.contains('ale')
kim_ner = kim_ner[exclude_kim_1 & exclude_kim_2 & exclude_kim_3 & exclude_kim_4 & exclude_kim_5 & exclude_kim_6 & exclude_kim_7 & exclude_kim_8 & exclude_kim_9]

/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [306]:
topic_modeling_df_dropped.iloc[iran_ner.article_idx.drop_duplicates()][topic_modeling_df_dropped['Topic'] == 'Military Nuclear Technology'].title

/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



1784     Mohammad Javad Zarif: Iran's foreign minister ...
3877     Iran hangs man on anti-gay charge - as Europe ...
4089     Iran claims it launched new cruise missile on ...
4603     Iran celebrates Islamic Revolution with threat...
4732     US Ambassador Grenell blasts Germany for celeb...
4862     Iran: Sabotage may be behind failed satellite ...
4864     Iran threatens retaliation over car bombing th...
4971     Iran unveils first semi-heavy missile-equipped...
5561             Iran Foreign Minister Javad Zarif resigns
5563                       Iran's foreign minister resigns
7263         Report: Iran merges military-affiliated banks
7882     Trump rips into US intel's Iran report, callin...
7912     Rouhani blames Iran’s ‘toughest economic situa...
8244     Iran starts 10-day festivities marking 40th an...
8299     Iran unveils, successfully tests new long-rang...
8457     Tehran ready to help rebuild Syria – Iran’s FM...
8463     Iran says it will never accept ‘humiliating co.

In [332]:
# Clean-up 'russia': 
russiagate_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('russiagate')]
russiagate_ner

,article_idx,ner_token,ner_label,std_ner_token
33438,9147.0,Russiagate,ORG,russiagate
38606,10759.0,Russiagate,PERSON,russiagate
39840,11113.0,Russiagate — RT World News,ORG,russiagate — rt world news
40174,11214.0,Russiagate,ORG,russiagate
40528,11324.0,Russiagate,ORG,russiagate


In [259]:
# Clean-up 'manafort': 
manafort_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('manafort')]
print(manafort_ner.std_ner_token.unique())  # Looks good

['paul manafort' 'manafort' 'manafort — rt usa news' "paul manafort's"]


In [344]:
# Clean-up 'northam' (Virginia Governor): 
northam_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('northam')]
northam_ner = northam_ner[~northam_ner.std_ner_token.str.contains('northampton')]
print(northam_ner.std_ner_token.unique()) # Looks good

['ralph northam' 'pam northam:' 'northam' "ralph northam's"
 'ralph northam’s' '’ northam']


In [263]:
# Clean-up 'blackface': 
blackface_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('blackface')]
print(blackface_ner.std_ner_token.unique()) # Looks good

['blackface' 'blackface’' 'virginia ‘blackface’']


In [264]:
# Clean-up 'fairfax': 
fairfax_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('fairfax')]
print(fairfax_ner.std_ner_token.unique()) # Looks good

['justin fairfax' 'fairfax' "justin fairfax's"]


In [408]:
# Clean-up 'isis': 
isis_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('isis')]
exclude_crisis_1 = ~ner_df_dropna.std_ner_token.str.contains('crisis')
isis_ner = isis_ner[exclude_crisis_1]
print(isis_ner.std_ner_token.unique()) # Looks good

['isis' 'post-isis' 'trump: isis' 'alabama isis' 'isis &']


/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [424]:
# Clean-up 'china': 
china_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('china')]
#exclude_crisis_1 = ~ner_df_dropna.std_ner_token.str.contains('crisis')
#isis_ner = isis_ner[exclude_crisis_1]
print(china_ner.std_ner_token.unique()) # Looks good

['china' 'china & turkey' 'china & russia' 'russia & china' 's. china sea'
 'airbus china' 'south china sea']


In [472]:
# Clean-up 'venezuela': 
venezuela_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('venezuela')]
print(venezuela_ner.std_ner_token.unique()) # Looks good

['venezuela' 'venezuelans' 'venezuelan' 'over’ venezuelan embassy'
 'sanctioned’ venezuelan' 'first venezuela' 'venezuela!’ bolton'
 'venezuela aid live' 'venezuelan fm' 'venezuelan vp'
 'starving venezuelans']


In [506]:
# Clean-up 'Super Bowl': 
superbowl_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('super bowl')]
print(superbowl_ner.std_ner_token.unique()) # Looks good

['the super bowl' 'super bowl' '5th super bowl' 'a super bowl'
 'rams super bowl' 'super bowl xxxvi' 'maroon 5 for super bowl'
 'super bowl halftime show' 'super bowl liii' 'super bowl lii'
 "super bowl'" '6 super bowl' 'sixth super bowl' 'super bowl tv'
 "trump's super bowl" 'another super bowl' 'this super bowl']


In [510]:
# Clean-up 'Syria': 
syria_ner = ner_df_dropna[ner_df_dropna.std_ner_token.str.contains('syria')]
print(syria_ner.std_ner_token.unique()) # Looks good

['syrian' 'syria' 'syrian army' 'syrian democratic forces' 'ne syria'
 'russian & syrian' 'the syrian democratic forces']


In [511]:
california_token_idx = ner_df[ner_df.ner_token.str.contains('California', na=False)].article_idx

# "Republican scandal":
cohen_token_idx = cohen_ner.article_idx.drop_duplicates()  # Excludes 'andy cohen' and 'richard cohen'
mueller_token_idx = mueller_ner.article_idx.drop_duplicates()  # Was pretty clean to start with
manafort_token_idx = manafort_ner.article_idx.drop_duplicates()  # Cleaned
russiagate_token_idx = russiagate_ner.article_idx.drop_duplicates()  # Very small 
repub_scandal_idx = pd.concat([cohen_token_idx, mueller_token_idx, manafort_token_idx, russiagate_token_idx]).drop_duplicates()

facebook_token_idx = ner_df[ner_df.ner_token.str.contains('Facebook', na=False)].article_idx

# Civil Rights:
smollett_token_idx = smollet_ner_3.article_idx.drop_duplicates()  # Already removed dupes
christchurch_token_idx = christchurch_ner.article_idx.drop_duplicates()  # This one is useless

# "National Security":
iran_token_idx = iran_ner.article_idx.drop_duplicates()  # Removed 'Miranda Lambert'
kim_token_idx = kim_ner.article_idx.drop_duplicates()  # Removed a lot of confounders
isis_token_idx = isis_ner.article_idx.drop_duplicates()
china_token_idx = china_ner.article_idx.drop_duplicates()
nat_sec_idx = pd.concat([iran_token_idx, kim_token_idx, china_token_idx, isis_token_idx]).drop_duplicates()

venezuela_token_idx = venezuela_ner.article_idx.drop_duplicates() 
syria_token_idx = syria_ner.article_idx.drop_duplicates()

# "Democrat Scandal":
virginia_token_idx = virginia_ner.article_idx.drop_duplicates()  # Excludes West Virginia
northam_token_idx = northam_ner.article_idx.drop_duplicates()
blackface_token_idx = blackface_ner.article_idx.drop_duplicates()
fairfax_token_idx = fairfax_ner.article_idx.drop_duplicates()
dems_scandal_idx = pd.concat([virginia_token_idx, northam_token_idx, blackface_token_idx, fairfax_token_idx]).drop_duplicates()

# "Sports"
superbowl_token_idx = superbowl_ner.article_idx.drop_duplicates()

In [ ]:
pd.merge(ner, df2, how='inner', on=['user_id'])

In [524]:
topic_modeling_df_dropped.iloc[facebook_token_idx].groupby(['Topic', 'source']).title.count()

Topic                                     source
Crime                                     bbc        1
                                          rt         1
Cybersecurity                             bbc        9
                                          cnn       14
                                          fox       20
                                          rt        14
Economy                                   bbc        2
                                          cnn        2
International Affairs (Latin America)     rt         1
Medical Research                          cnn        2
Political Investigation (Cohen/Virginia)  bbc       11
                                          cnn       11
                                          fox       13
                                          rt        22
Religious Intolerance                     fox        1
                                          rt         1
Social Media                              bbc        2
                

#### NER-based means:

In [358]:
ttests.head()

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
0,CNN,BBC,Political Investigation (Cohen/Virginia),Unknown,0.012947,143.463212,0.0,548537.03,0.2564
1,CNN,BBC,Political Investigation (Cohen/Virginia),Sports,-0.017921,-28.569813,0.0,15149.28,-0.2881
2,CNN,BBC,Political Investigation (Cohen/Virginia),Military Nuclear Technology,0.065761,21.615786,0.0,167.23,1.5476
3,CNN,BBC,Political Investigation (Cohen/Virginia),Film Industry,-0.007157,-15.426710,0.0,18759.83,-0.1334
4,CNN,BBC,Political Investigation (Cohen/Virginia),Cybersecurity,0.002196,5.865892,0.0,19802.31,0.0464


"Republican Scandal" in 'Political Investigation (Cohen/Virginia)'

vs 

"National Security" in 'Military Nuclear Technology'

In [604]:
from statsmodels.stats.weightstats import ttest_ind

ttests_ner = pd.DataFrame(columns=['Baseline Source', 'Target Source', 'Baseline Topic - NER Token', 
                               'Target Topic - NER Token', 'Mean Difference',
                               'Welsh\'s t-Statistic', 'Two-Tailed P-Value', 
                               'Satterthwait DoF', 'Cohen\'s d'])

In [605]:
# 0
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'China'
target_ner_df = topic_modeling_df_dropped.iloc[china_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'bbc'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 12  # Economy
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(1308,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(352,)
TARGET TOPIC: Economy


In [606]:
# 1
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'China'
target_ner_df = topic_modeling_df_dropped.iloc[china_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'fox'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 12  # Economy
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(3924,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(48,)
TARGET TOPIC: Economy


In [607]:
# 2
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'China'
target_ner_df = topic_modeling_df_dropped.iloc[china_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'rt'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 12  # Economy
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(1962,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(560,)
TARGET TOPIC: Economy


In [608]:
# 3
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'Venezuela'
target_ner_df = topic_modeling_df_dropped.iloc[venezuela_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'bbc'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 17  # International Affairs (Latin America)
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(1308,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(810,)
TARGET TOPIC: International Affairs (Latin America)


In [609]:
# 4
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'Venezuela'
target_ner_df = topic_modeling_df_dropped.iloc[venezuela_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'fox'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 17  # International Affairs (Latin America)
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(3924,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(1740,)
TARGET TOPIC: International Affairs (Latin America)


In [610]:
# 5
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'Venezuela'
target_ner_df = topic_modeling_df_dropped.iloc[venezuela_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'rt'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 17  # International Affairs (Latin America)
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(1962,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(5010,)
TARGET TOPIC: International Affairs (Latin America)


In [611]:
# 6
baseline_ner_token = 'Virginia, Northam, blackface, Fairfax'
baseline_ner_df = topic_modeling_df_dropped.iloc[dems_scandal_idx]

target_ner_token = 'China'
target_ner_df = topic_modeling_df_dropped.iloc[china_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'bbc'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 12  # Economy
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(248,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(352,)
TARGET TOPIC: Economy


In [612]:
# 7
baseline_ner_token = 'Virginia, Northam, blackface, Fairfax'
baseline_ner_df = topic_modeling_df_dropped.iloc[dems_scandal_idx]

target_ner_token = 'China'
target_ner_df = topic_modeling_df_dropped.iloc[china_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'fox'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 12  # Economy
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(713,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(48,)
TARGET TOPIC: Economy


In [613]:
# 8
baseline_ner_token = 'Virginia, Northam, blackface, Fairfax'
baseline_ner_df = topic_modeling_df_dropped.iloc[dems_scandal_idx]

target_ner_token = 'China'
target_ner_df = topic_modeling_df_dropped.iloc[china_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'rt'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 12  # Economy
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(279,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(560,)
TARGET TOPIC: Economy


In [614]:
# 9
baseline_ner_token = 'Virginia, Northam, blackface, Fairfax'
baseline_ner_df = topic_modeling_df_dropped.iloc[dems_scandal_idx]

target_ner_token = 'Venezuela'
target_ner_df = topic_modeling_df_dropped.iloc[venezuela_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'bbc'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 17  # International Affairs (Latin America)
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(248,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(810,)
TARGET TOPIC: International Affairs (Latin America)


In [615]:
# 10
baseline_ner_token = 'Virginia, Northam, blackface, Fairfax'
baseline_ner_df = topic_modeling_df_dropped.iloc[dems_scandal_idx]

target_ner_token = 'Venezuela'
target_ner_df = topic_modeling_df_dropped.iloc[venezuela_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'fox'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 17  # International Affairs (Latin America)
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(713,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(1740,)
TARGET TOPIC: International Affairs (Latin America)


In [616]:
# 11
baseline_ner_token = 'Virginia, Northam, blackface, Fairfax'
baseline_ner_df = topic_modeling_df_dropped.iloc[dems_scandal_idx]

target_ner_token = 'Venezuela'
target_ner_df = topic_modeling_df_dropped.iloc[venezuela_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'rt'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 17  # International Affairs (Latin America)
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(279,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(5010,)
TARGET TOPIC: International Affairs (Latin America)


In [617]:
# 12
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'Facebook'
target_ner_df = topic_modeling_df_dropped.iloc[facebook_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'bbc'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 5  # Cybersecurity
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(1308,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(126,)
TARGET TOPIC: Cybersecurity


In [618]:
# 13
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'Facebook'
target_ner_df = topic_modeling_df_dropped.iloc[facebook_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'fox'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 5  # Cybersecurity
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(3924,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(280,)
TARGET TOPIC: Cybersecurity


In [619]:
# 14
baseline_ner_token = 'Mueller, Cohen, Manafort, Russiagate'
baseline_ner_df = topic_modeling_df_dropped.iloc[repub_scandal_idx]

target_ner_token = 'Facebook'
target_ner_df = topic_modeling_df_dropped.iloc[facebook_token_idx]

baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (baseline_ner_df.source == baseline_source)
target_source = 'rt'
target_source_title = target_source.upper()
baseline_target_filt = (baseline_ner_df.source == target_source)

baseline_source_df = baseline_ner_df[baseline_source_filt]
target_source_df = baseline_ner_df[baseline_target_filt]
                                               
baseline_topic_num = 2  # Political Investigation (Cohen/Virginia)
baseline_topic = topic_translation[baseline_topic_num]
baseline_topic_filt = (baseline_ner_df.Dominant_Topic == baseline_topic_num)
baseline_topic_df = baseline_ner_df[baseline_topic_filt]

baseline_topic_baseline_source_filt = (baseline_topic_df.source == baseline_source)
baseline_topic_baseline_source_df = baseline_topic_df[baseline_topic_baseline_source_filt]
baseline_topic_baseline_source_idx = baseline_topic_baseline_source_df.index

baseline_topic_target_source_filt = (baseline_topic_df.source == target_source)
baseline_topic_target_source_df = baseline_topic_df[baseline_topic_target_source_filt]
baseline_topic_target_source_idx = baseline_topic_target_source_df.index

baseline_topic_dist = cos_sim_mat[baseline_topic_baseline_source_idx, :][:, baseline_topic_target_source_idx]
baseline_topic_dist_arr = baseline_topic_dist.flatten()
print(baseline_topic_dist_arr.shape)
baseline_mean = baseline_topic_dist_arr.mean()

print('BASELINE TOPIC:', baseline_topic)

target_topic_num = 5  # Cybersecurity
target_topic = topic_translation[target_topic_num]
target_topic_filt = (target_ner_df.Dominant_Topic == target_topic_num)
target_topic_df = target_ner_df[target_topic_filt]

target_topic_baseline_source_filt = (target_topic_df.source == baseline_source)
target_topic_baseline_source_df = target_topic_df[target_topic_baseline_source_filt]
target_topic_baseline_source_idx = target_topic_baseline_source_df.index

target_topic_target_source_filt = (target_topic_df.source == target_source)
target_topic_target_source_df = target_topic_df[target_topic_target_source_filt]
target_topic_target_source_idx = target_topic_target_source_df.index

target_topic_dist = cos_sim_mat[target_topic_baseline_source_idx, :][:, target_topic_target_source_idx]
target_topic_dist_arr = target_topic_dist.flatten()
print(target_topic_dist_arr.shape)
target_mean = target_topic_dist_arr.mean()

print('TARGET TOPIC:', target_topic)

diff_in_means = baseline_mean - target_mean
tstat, pvalue, df = ttest_ind(baseline_topic_dist_arr, 
                              target_topic_dist_arr, 
                              usevar='unequal')
cohens_d = cohend(baseline_topic_dist_arr, target_topic_dist_arr)

a_row = {'Baseline Source': baseline_source_title, 
         'Target Source': target_source_title, 
         'Baseline Topic - NER Token': baseline_topic + ' - '+ baseline_ner_token, 
         'Target Topic - NER Token': target_topic + ' - '+ target_ner_token, 
         'Mean Difference': round(diff_in_means, 6),
         'Welsh\'s t-Statistic': round(tstat, 6),
         'Two-Tailed P-Value': round(pvalue, 6),
         'Satterthwait DoF': round(df, 2),
         'Cohen\'s d': round(cohens_d, 4)
        }
ttests_ner = ttests_ner.append(a_row, ignore_index=True)

(1962,)
BASELINE TOPIC: Political Investigation (Cohen/Virginia)
(196,)
TARGET TOPIC: Cybersecurity


In [620]:
ttests_ner

,Baseline Source,Target Source,Baseline Topic - NER Token,Target Topic - NER Token,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
0,CNN,BBC,Political Investigation (Cohen/Virginia) - Mue...,Economy - China,-0.028899,-10.501721,0.000000,455.47,-0.6843
1,CNN,FOX,Political Investigation (Cohen/Virginia) - Mue...,Economy - China,-0.044667,-6.024262,0.000000,47.58,-1.0048
2,CNN,RT,Political Investigation (Cohen/Virginia) - Mue...,Economy - China,-0.067920,-29.178907,0.000000,694.15,-1.5527
3,CNN,BBC,Political Investigation (Cohen/Virginia) - Mue...,International Affairs (Latin America) - Venezuela,0.015410,11.590946,0.000000,2054.35,0.4998
4,CNN,FOX,Political Investigation (Cohen/Virginia) - Mue...,International Affairs (Latin America) - Venezuela,0.003331,3.224924,0.001272,3406.48,0.0925
5,CNN,RT,Political Investigation (Cohen/Virginia) - Mue...,International Affairs (Latin America) - Venezuela,0.010050,11.382216,0.000000,3530.23,0.3042
6,CNN,BBC,Political Investigation (Cohen/Virginia) - Vir...,Economy - China,-0.055828,-18.124913,0.000000,568.66,-1.4314
7,CNN,FOX,Political Investigation (Cohen/Virginia) - Vir...,Economy - China,-0.055198,-7.337440,0.000000,50.42,-1.2315
8,CNN,RT,Political Investigation (Cohen/Virginia) - Vir...,Economy - China,-0.086017,-29.154134,0.000000,793.41,-1.9754
9,CNN,BBC,Political Investigation (Cohen/Virginia) - Vir...,International Affairs (Latin America) - Venezuela,-0.011518,-6.002542,0.000000,399.94,-0.4390


In [603]:
0.05 / 15

0.0033333333333333335

In [621]:
ttests_ner.to_csv('data/processed/2019_05_02_ttests_ner.csv')

In [456]:
ttests[ttests['Target Topic'] == 'Economy']

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
11,CNN,BBC,Political Investigation (Cohen/Virginia),Economy,-0.001623,-8.770962,0.0,107573.67,-0.0359
33,CNN,FOX,Political Investigation (Cohen/Virginia),Economy,-0.026007,-67.933045,0.0,26415.46,-0.4599
55,CNN,RT,Political Investigation (Cohen/Virginia),Economy,-0.029282,-121.531289,0.0,67420.84,-0.5890


In [470]:
ttests

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
0,CNN,BBC,Political Investigation (Cohen/Virginia),Unknown,0.012947,143.463212,0.000000,548537.03,0.2564
1,CNN,BBC,Political Investigation (Cohen/Virginia),Sports,-0.017921,-28.569813,0.000000,15149.28,-0.2881
2,CNN,BBC,Political Investigation (Cohen/Virginia),Military Nuclear Technology,0.065761,21.615786,0.000000,167.23,1.5476
3,CNN,BBC,Political Investigation (Cohen/Virginia),Film Industry,-0.007157,-15.426710,0.000000,18759.83,-0.1334
4,CNN,BBC,Political Investigation (Cohen/Virginia),Cybersecurity,0.002196,5.865892,0.000000,19802.31,0.0464
5,CNN,BBC,Political Investigation (Cohen/Virginia),Entertainment Industry,-0.035410,-9.499398,0.000000,395.36,-0.5760
6,CNN,BBC,Political Investigation (Cohen/Virginia),Health,-0.010577,-9.868597,0.000000,2315.13,-0.2188
7,CNN,BBC,Political Investigation (Cohen/Virginia),Brexit,0.061744,99.733987,0.000000,4835.41,1.4124
8,CNN,BBC,Political Investigation (Cohen/Virginia),Space Industry,-0.006062,-10.397575,0.000000,11695.03,-0.1124
9,CNN,BBC,Political Investigation (Cohen/Virginia),US Campaign Politics,0.036713,53.103377,0.000000,4627.22,0.8027


In [348]:
topic_modeling_df_dropped.iloc[repub_scandal_idx][topic_modeling_df_dropped.Topic == 'International Affairs (Latin America)'].title

/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



13445    News today: Oscar Takeways, North Korea Summit...
3198     Trump hints at payback for 'evil' enemies over...
3202     Viewpoint: Mueller called it (largely) right o...
3215     Donald Trump launches furious attack on Robert...
4818     Sarah Sanders says she ‘voluntarily’ agreed to...
6963     Dems' conference call grapples with Mueller re...
7035     Donna Brazile: What we really need to learn fr...
7036     Rep. Doug Collins on Mueller report summary: '...
7037     Pelosi, Schumer, other top Dems question impar...
7039     Rep. Al Green after Mueller findings released:...
7040     Reporter's Notebook: Mueller probe findings tr...
7041     Pence: Mueller findings 'should be welcomed by...
7042     George Conway hits back at Sarah Sanders after...
7044     James Comey tweets forest photo after Mueller ...
7096     CNN’s Chris Cuomo lectures Trump supporters fo...
8035     Stone pleads not guilty to obstruction, lying ...
8076     Mueller probe: Fully armed FBI agents arrest T.

In [42]:
def standardize_ner(some_str):
    """
    TODO: Finish docstring
    """
    rv = some_str.lower().strip()
    return rv

In [48]:
std_ner_token = ner_df[pd.notnull(ner_df.ner_token)].ner_token.apply(standardize_ner)

In [49]:
ner_df['std_ner_token'] = std_ner_token

In [52]:
counts_ner_2 = ner_df.groupby('std_ner_token').count()

In [74]:
counts_ner_2[counts_ner_2['ner_label'] > 100]

,article_idx,ner_token,ner_label
std_ner_token,,,
2019,109,109,109
2020,168,168,168
brexit,317,317,317
california,137,137,137
china,199,199,199
democrats,116,116,116
eu,151,151,151
facebook,169,169,169
first,172,172,172


'Mueller':

In [103]:
mueller_idx = ner_df[ner_df.std_ner_token == 'mueller'].article_idx
mueller_idx.shape

(128,)

In [108]:
topic_modeling_df_dropped.iloc[mueller_idx].groupby(['source', 'Topic']).title.count()

source  Topic                                   
bbc     International Affairs (Latin America)        2
        Political Investigation (Cohen/Virginia)     3
        Unknown                                      1
cnn     International Affairs (Latin America)       19
        Political Investigation (Cohen/Virginia)    38
        Unknown                                     10
fox     International Affairs (Latin America)       10
        Political Investigation (Cohen/Virginia)    17
        Unknown                                      8
rt      International Affairs (Latin America)        8
        Political Investigation (Cohen/Virginia)    10
        Unknown                                      2
Name: title, dtype: int64

Looks like mostly CNN and Political Investigation

In [109]:
mueller_df = topic_modeling_df_dropped.iloc[mueller_idx]

In [119]:
mueller_cnn_topic2_idx = mueller_df[(mueller_df.source == 'cnn') & (mueller_df.Dominant_Topic == 2)].index

Trump:

In [123]:
trump_idx = ner_df[ner_df.std_ner_token == 'trump'].article_idx
print(trump_idx.shape)
topic_modeling_df_dropped.iloc[trump_idx].groupby(['source', 'Topic']).title.count()

(828,)


source  Topic                                             
bbc     Economy                                                 9
        Film Industry                                           1
        International Affairs (Latin America)                  20
        International Affairs (Middle East and South Asia)      2
        Medical Research                                        1
        Political Investigation (Cohen/Virginia)               15
        US Campaign Politics                                    3
        Unknown                                                 9
cnn     Corporate News                                          1
        Cybersecurity                                           5
        Economy                                                16
        Emergencies                                             2
        International Affairs (Latin America)                  71
        International Affairs (Middle East and South Asia)     10
        Medical R

Let's take RT, FOX, CNN for International Affairs (Latin America) topic:

In [131]:
trump_df = topic_modeling_df_dropped.iloc[trump_idx].drop_duplicates()

In [134]:
trump_cnn_topic17_idx = trump_df[(trump_df.source == 'cnn') & (mueller_df.Dominant_Topic == 17)]#.index
trump_fox_topic17_idx = trump_df[(trump_df.source == 'fox') & (mueller_df.Dominant_Topic == 17)]#.index
trump_rt_topic17_idx = trump_df[(trump_df.source == 'rt') & (mueller_df.Dominant_Topic == 17)]#.index

/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [137]:
trump_cnn_topic17_idx#.source

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords,Topic,ner_tokens,ner_entities
14821,NaN,NaN,Trump's evening for the ages: When he learned ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Washington (CNN) On the evening Robert Mueller...,17.0,0.2846,"trump, president, congress, white_house, admin...",International Affairs (Latin America),"(Trump, evening, Mueller)","['ORG', 'TIME', 'PERSON']"
14868,NaN,NaN,Trump claims vindication after Mueller finds n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,(CNN) A triumphant President Donald Trump emer...,17.0,0.2756,"trump, president, congress, white_house, admin...",International Affairs (Latin America),"(Trump, Mueller)","['ORG', 'PERSON']"
14878,NaN,NaN,Mueller report did not find Trump or his campa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Washington (CNN) Special counsel Robert Muelle...,17.0,0.4042,"trump, president, congress, white_house, admin...",International Affairs (Latin America),"(Mueller, Trump, Russia)","['PERSON', 'ORG', 'GPE']"
14975,NaN,NaN,Mueller report: Trump moves to weaponize findings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"Washington (CNN) For more than two years, Dona...",17.0,0.2801,"trump, president, congress, white_house, admin...",International Affairs (Latin America),"(Mueller, Trump)","['PERSON', 'ORG']"
15157,NaN,NaN,Mueller report may just be the beginning of Tr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,(CNN) The impeachment of President Donald Trum...,17.0,0.2967,"trump, president, congress, white_house, admin...",International Affairs (Latin America),"(Mueller, Trump)","['PERSON', 'ORG']"


### Vectorization:

In [ ]:
topic_modeling_df_dropped = pd.read_csv('data/processed/2019_04_26_AllWCNN_WTopics_noemptystr_inttopics.csv')

In [107]:
%%time
# https://stackoverflow.com/questions/42094180/spacy-how-to-load-google-news-word2vec-vectors
import gensim
import spacy

# Path to google news vectors
google_news_path = "data/word_embeddings/GoogleNews-vectors-negative300.bin.gz"

# Load google news vecs in gensim
model = gensim.models.KeyedVectors.load_word2vec_format(google_news_path, binary=True)

# Init blank english spacy nlp object
nlp = spacy.blank('en')

# Loop through range of all indexes, get words associated with each index.
# The words in the keys list will correspond to the order of the google embed matrix
keys = []
for idx in range(3000000):
    keys.append(model.index2word[idx])

# Set the vectors for our nlp object to the google news vectors
nlp.vocab.vectors = spacy.vocab.Vectors(data=model.syn0, keys=keys)

CPU times: user 2min 50s, sys: 7.98 s, total: 2min 58s
Wall time: 3min


/Users/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning:

Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).



In [108]:
%%time
# Vectorize texts:
vectorized_texts = nlp(topic_modeling_df_dropped['clean_text'][0]).vector
idx = 1
for text in topic_modeling_df_dropped[1:]['clean_text']:
    vector = nlp(text).vector
    vectorized_texts = np.vstack([vectorized_texts, vector])

KeyboardInterrupt: 

Just checking the cosine distances make sense:

In [112]:
from scipy.spatial.distance import cosine

In [123]:
cosine(vectorized_texts[13000], vectorized_texts[5])


0.17286449670791626

In [124]:
cos_sim_mat[13000][5]

0.17286432

[End of cosine distance check]

In [ ]:
%%time
# Calculate cosine similarity matrix (n_jobs = -1 means use all CPU cores):
cos_sim_mat = (pairwise_distances(vectorized_texts, metric='cosine', n_jobs = -1))

In [ ]:
# Save Cosine dissimilarity matrix:
np.save('data/processed/2019_04_19_cosine_similarity_matrix', cos_sim_mat)

In [55]:
# Load Cosine dissimilarity matrix:
cos_sim_mat = np.load('data/processed/2019_04_19_cosine_similarity_matrix.npy')

In [13]:
cos_sim_mat

array([[2.3841858e-07, 1.0846871e-01, 7.6667786e-02, ..., 1.4119041e-01,
        1.5211523e-01, 1.2163323e-01],
       [1.0846871e-01, 0.0000000e+00, 1.6264653e-01, ..., 2.5990212e-01,
        2.7582240e-01, 2.6790547e-01],
       [7.6667786e-02, 1.6264653e-01, 0.0000000e+00, ..., 1.5324438e-01,
        1.5492499e-01, 1.4850461e-01],
       ...,
       [1.4119041e-01, 2.5990212e-01, 1.5324438e-01, ..., 0.0000000e+00,
        8.7890983e-02, 1.0817826e-01],
       [1.5211523e-01, 2.7582240e-01, 1.5492499e-01, ..., 8.7890983e-02,
        1.7881393e-07, 9.4714403e-02],
       [1.2163317e-01, 2.6790571e-01, 1.4850473e-01, ..., 1.0817826e-01,
        9.4714522e-02, 0.0000000e+00]], dtype=float32)

In [67]:
def make_bias_df(cnn_idx_arr, target_source_idx_arr, target_article_category, topic, cos_sim_mat):
    """
    """
    similarity_df = pd.DataFrame(columns=['cnn_article', 'target_article', 
                                          'target_article_category', 'topic', 'cos_similarity'])
    outer_loop = 0
    for cnn_idx in cnn_idx_arr:
        print('OUTER LOOP', outer_loop)
        for target_idx in target_source_idx_arr:
            row = {'cnn_article': cnn_idx,
                   'target_article': target_idx,
                   'target_article_category': target_article_category,
                   'topic': topic,
                   'cos_similarity': cos_sim_mat[cnn_idx, target_idx]}
            similarity_df = similarity_df.append(row, ignore_index=True)
        outer_loop += 1
    return similarity_df

def make_bias_df(cnn_idx_arr, target_source_idx_arr, target_article_category, topic, cos_sim_mat):
    """
    """
    similarity_df = pd.DataFrame(columns=['target_article_category', 'topic', 'cos_similarity'])
    filtered = cos_sim_mat[cnn_idx_arr, :][:, target_source_idx_arr]
    similarity_df['cos_similarity'] = filtered.flatten()
    similarity_df['topic'] = topic
    similarity_df['target_article_category'] = target_article_category
#     outer_loop = 0
#     for cnn_idx in cnn_idx_arr:
#         print('OUTER LOOP', outer_loop)
#         for target_idx in target_source_idx_arr:
#             row = {'cnn_article': cnn_idx,
#                    'target_article': target_idx,
#                    'target_article_category': target_article_category,
#                    'topic': topic,
#                    'cos_similarity': cos_sim_mat[cnn_idx, target_idx]}
#             similarity_df = similarity_df.append(row, ignore_index=True)
#         outer_loop += 1
    return similarity_df

In [11]:
topics = topic_modeling_df_dropped.Topic.unique()
sources = topic_modeling_df_dropped.source.unique()
baseline_source = 'cnn'

In [ ]:
cos_bias_df = pd.DataFrame(columns=['Topic', 'Baseline Source', 'Baseline Article Index', 'Target Source', 'Target Source Article Index', 'Bias'])

In [66]:
all_topic_matrices = []
all_topic_indx = np.arange(0, 23)
print(all_topic_indx)
for topic_idx in all_topic_indx:
    topic_filter = (topic_modeling_df_dropped.Dominant_Topic == all_topic_indx[topic_idx])
    idx_thistopic = topic_modeling_df_dropped[topic_filter].index.values
    
    baseline_filter = (topic_modeling_df_dropped.source == baseline_source)
    othersource_filter = (topic_modeling_df_dropped.source != baseline_source)
    
    idx_baseline = topic_modeling_df_dropped[topic_filter & baseline_filter].index.values
    idx_other = topic_modeling_df_dropped[topic_filter & othersource_filter].index.values
    
    baseline_mat
    
#     for current_topic_idx in idx_thistopic:
#         other_sources_idxs = 
#         print(current_topic_idx)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


NameError: name 'baseline_mat' is not defined

In [ ]:
topic_filter = (topic_modeling_df_dropped.Dominant_Topic == 0)
idx_thistopic = topic_modeling_df_dropped[topic_filter].index.values
baseline_filter = (topic_modeling_df_dropped.source != baseline_source)
idx_baseline = topic_modeling_df_dropped[topic_filter & baseline_filter].index.values


In [68]:
topic_translation = {0: 'Unknown',
                     1: 'Sports',
                     2: 'Political Scandal (Cohen/Virginia)',
                     3: 'Military Nuclear Technology',
                     4: 'Film Industry',
                     5: 'Cybersecurity',
                     6: 'Entertainment Industry',
                     7: 'Health',
                     8: 'Brexit',
                     9: 'Space Industry',
                     10: 'US Campaign Politics',
                     11: 'Medical Research',
                     12: 'Economy',
                     13: 'Emergencies',
                     14: 'Religious Intolerance',
                     15: 'Family Affairs',
                     16: 'International Affairs (Middle East and South Asia)',
                     17: 'International Affairs (Latin America)',
                     18: 'Corporate News',
                     19: 'Fashion/Issues of Race',
                     20: 'Crime',
                     21: 'Social Media',
                     22: 'Nature and Environment'}

In [69]:
%%time
chosen_topics = range(23) #[1, 2, 10, 22] # [1, 2, 10, 12, 13, 15, 18]
all_dfs = []
baseline_source = 'cnn'
for topic_num in chosen_topics:
    print('TOPIC NUM', topic_num)
    topic_filter = (topic_modeling_df_dropped.Dominant_Topic == topic_num)
    idx_thistopic = topic_modeling_df_dropped[topic_filter].index.values
    
    baseline_filter = (topic_modeling_df_dropped.source == baseline_source)
    othersource_filter = (topic_modeling_df_dropped.source != baseline_source)
    
    idx_baseline = topic_modeling_df_dropped[topic_filter & baseline_filter].index.values
    idx_other = topic_modeling_df_dropped[topic_filter & othersource_filter].index.values

    other_sources_df = topic_modeling_df_dropped[topic_filter & othersource_filter]
    for source in other_sources_df.source.unique():
        print('SOURCE:', source)
        other_source_filter = (other_sources_df.source == source)
        other_source_df = other_sources_df[other_source_filter]  # Other source in this topic
        
        other_source_idx = other_source_df.index.values

        bias_df = make_bias_df(cnn_idx_arr=idx_baseline, 
                           target_source_idx_arr=other_source_idx, 
                           target_article_category=source,
                           topic=topic_num, 
                           cos_sim_mat=cos_sim_mat)
        print(len(bias_df))
        all_dfs.append(bias_df)

TOPIC NUM 0
SOURCE: bbc
1606225
SOURCE: fox
1655437
SOURCE: rt
799695
TOPIC NUM 1
SOURCE: bbc
14673
SOURCE: fox
11859
SOURCE: rt
20234
TOPIC NUM 2
SOURCE: bbc
329175
SOURCE: fox
454608
SOURCE: rt
317394
TOPIC NUM 3
SOURCE: bbc
168
SOURCE: fox
432
SOURCE: rt
1152
TOPIC NUM 4
SOURCE: bbc
17688
SOURCE: fox
12328
SOURCE: rt
4020
TOPIC NUM 5
SOURCE: bbc
18078
SOURCE: fox
20960
SOURCE: rt
15589
TOPIC NUM 6
SOURCE: bbc
396
SOURCE: fox
396
SOURCE: rt
44
TOPIC NUM 7
SOURCE: bbc
2291
SOURCE: fox
9006
SOURCE: rt
1501
TOPIC NUM 8
SOURCE: bbc
4680
SOURCE: fox
780
SOURCE: rt
6720
TOPIC NUM 9
SOURCE: bbc
11270
SOURCE: fox
12190
SOURCE: rt
21620
TOPIC NUM 10
SOURCE: bbc
4508
SOURCE: fox
23667
SOURCE: rt
10465
TOPIC NUM 11
SOURCE: bbc
13804
SOURCE: fox
8925
SOURCE: rt
2856
TOPIC NUM 12
SOURCE: bbc
72688
SOURCE: fox
24072
SOURCE: rt
51684
TOPIC NUM 13
SOURCE: bbc
17017
SOURCE: fox
23205
SOURCE: rt
21182
TOPIC NUM 14
SOURCE: bbc
2184
SOURCE: fox
3150
SOURCE: rt
6216
TOPIC NUM 15
SOURCE: bbc
525
SOURCE: f

In [70]:
dfs_noNER = pd.concat(all_dfs)

In [71]:
dfs_noNER

,target_article_category,topic,cos_similarity
0,bbc,0,0.174427
1,bbc,0,0.190313
2,bbc,0,0.124061
3,bbc,0,0.194349
4,bbc,0,0.158418
5,bbc,0,0.148662
6,bbc,0,0.167092
7,bbc,0,0.157454
8,bbc,0,0.125403
9,bbc,0,0.170594


In [127]:
dfs_noNER.groupby(['target_article_category', 'topic']).describe()

cos_similarity                                \
                                       count      mean       std       min   
target_article_category topic                                                
bbc                     0          1606225.0  0.138262  0.055117  0.012507   
                        1            14673.0  0.169127  0.075379  0.020208   
                        2           329175.0  0.151207  0.045357  0.013860   
                        3              168.0  0.085444  0.039419  0.018555   
                        4            17688.0  0.158362  0.060801  0.018631   
                        5            18078.0  0.149009  0.049191  0.017813   
                        6              396.0  0.186615  0.074162  0.020308   
                        7             2291.0  0.161782  0.051160  0.014094   
                        8             4680.0  0.089461  0.042006  0.011126   
                        9            11270.0  0.157267  0.061320  0.016843   
                        10            4508.0  0.114493  0.046113  0.018553   
                        11           13804.0  0.142463  0.045777  0.015448   
                        12           72688.0  0.152829  0.045116  0.017159   
                        13           17017.0  0.184375  0.055756  0.014398   
                        14            2184.0  0.146681  0.048939  0.022488   
                        15             525.0  0.176612  0.069728  0.025663   
                        16            7872.0  0.143880  0.053192  0.012621   
                        17           21692.0  0.135338  0.058029  0.017720   
                        18            2904.0  0.172837  0.056254  0.024855   
                        19             420.0  0.210490  0.080451  0.020270   
                        20           12700.0  0.145195  0.051937  0.011945   
                        21             484.0  0.108983  0.042477  0.021713   
                        22            6004.0  0.158966  0.052023  0.020991   
fox                     0          1655437.0  0.159065  0.056019  0.012510   
                        1            11859.0  0.191606  0.057559  0.020436   
                        2           454608.0  0.163898  0.055034  0.011135   
                        3              432.0  0.127368  0.060507  0.022228   
                        4            12328.0  0.181748  0.060160  0.021455   
                        5            20960.0  0.171809  0.060553  0.019468   
                        6              396.0  0.229133  0.092563  0.029837   
...                                      ...       ...       ...       ...   
                        16           13632.0  0.178648  0.060140  0.018068   
                        17           43945.0  0.153509  0.062778  0.013994   
                        18            1628.0  0.232819  0.072836  0.034637   
                        19             441.0  0.216871  0.061610  0.052438   
                        20           35100.0  0.159640  0.055899  0.016398   
                        21             418.0  0.135560  0.069407  0.037710   
                        22           12640.0  0.180010  0.057801  0.012867   
rt                      0           799695.0  0.151197  0.051857  0.014611   
                        1            20234.0  0.160198  0.052404  0.017078   
                        2           317394.0  0.155853  0.048168  0.015631   
                        3             1152.0  0.148380  0.067370  0.028473   
                        4             4020.0  0.172320  0.059216  0.029250   
                        5            15589.0  0.165651  0.053812  0.022826   
                        6               44.0  0.232860  0.042558  0.150238   
                        7             1501.0  0.176891  0.057879  0.036963   
                        8             6720.0  0.101983  0.051413  0.018212   
                        9            21620.0  0.180254  0.065560  0.015229   
                        10           10465.0  0.1224

Output summary statistics for mean cosine distance between CNN and Target Article Source by topic:

In [133]:
cos_sim_summary_tbl = dfs_noNER.groupby(['target_article_category', 'topic']).describe()

In [136]:
cos_sim_summary_tbl

cos_similarity                                \
                                       count      mean       std       min   
target_article_category topic                                                
bbc                     0          1606225.0  0.138262  0.055117  0.012507   
                        1            14673.0  0.169127  0.075379  0.020208   
                        2           329175.0  0.151207  0.045357  0.013860   
                        3              168.0  0.085444  0.039419  0.018555   
                        4            17688.0  0.158362  0.060801  0.018631   
                        5            18078.0  0.149009  0.049191  0.017813   
                        6              396.0  0.186615  0.074162  0.020308   
                        7             2291.0  0.161782  0.051160  0.014094   
                        8             4680.0  0.089461  0.042006  0.011126   
                        9            11270.0  0.157267  0.061320  0.016843   
                        10            4508.0  0.114493  0.046113  0.018553   
                        11           13804.0  0.142463  0.045777  0.015448   
                        12           72688.0  0.152829  0.045116  0.017159   
                        13           17017.0  0.184375  0.055756  0.014398   
                        14            2184.0  0.146681  0.048939  0.022488   
                        15             525.0  0.176612  0.069728  0.025663   
                        16            7872.0  0.143880  0.053192  0.012621   
                        17           21692.0  0.135338  0.058029  0.017720   
                        18            2904.0  0.172837  0.056254  0.024855   
                        19             420.0  0.210490  0.080451  0.020270   
                        20           12700.0  0.145195  0.051937  0.011945   
                        21             484.0  0.108983  0.042477  0.021713   
                        22            6004.0  0.158966  0.052023  0.020991   
fox                     0          1655437.0  0.159065  0.056019  0.012510   
                        1            11859.0  0.191606  0.057559  0.020436   
                        2           454608.0  0.163898  0.055034  0.011135   
                        3              432.0  0.127368  0.060507  0.022228   
                        4            12328.0  0.181748  0.060160  0.021455   
                        5            20960.0  0.171809  0.060553  0.019468   
                        6              396.0  0.229133  0.092563  0.029837   
...                                      ...       ...       ...       ...   
                        16           13632.0  0.178648  0.060140  0.018068   
                        17           43945.0  0.153509  0.062778  0.013994   
                        18            1628.0  0.232819  0.072836  0.034637   
                        19             441.0  0.216871  0.061610  0.052438   
                        20           35100.0  0.159640  0.055899  0.016398   
                        21             418.0  0.135560  0.069407  0.037710   
                        22           12640.0  0.180010  0.057801  0.012867   
rt                      0           799695.0  0.151197  0.051857  0.014611   
                        1            20234.0  0.160198  0.052404  0.017078   
                        2           317394.0  0.155853  0.048168  0.015631   
                        3             1152.0  0.148380  0.067370  0.028473   
                        4             4020.0  0.172320  0.059216  0.029250   
                        5            15589.0  0.165651  0.053812  0.022826   
                        6               44.0  0.232860  0.042558  0.150238   
                        7             1501.0  0.176891  0.057879  0.036963   
                        8             6720.0  0.101983  0.051413  0.018212   
                        9            21620.0  0.180254  0.065560  0.015229   
                        10           10465.0  0.1224

In [130]:
dfs_noNER.groupby(['target_article_category', 'topic']).describe().to_csv('data/processed/2019_04_30_CNNvsOthersDistanceSummary.csv', index=True)

Check that the means make sense:

In [145]:
topic_modeling_df_dropped.source.unique()

array(['bbc', 'fox', 'rt', 'cnn'], dtype=object)

In [183]:
cnn_topic5_filt = ((topic_modeling_df_dropped.Dominant_Topic == 5) & (topic_modeling_df_dropped.source == 'cnn'))
bbc_topic5_filt = ((topic_modeling_df_dropped.Dominant_Topic == 5) & (topic_modeling_df_dropped.source == 'bbc'))
cnn_topic5_filt_idx = topic_modeling_df_dropped[cnn_topic5_filt].index
bbc_topic5_filt_idx = topic_modeling_df_dropped[bbc_topic5_filt].index
filtered_cos_mat = cos_sim_mat[cnn_topic5_filt_idx, :][:, bbc_topic5_filt_idx]
filtered_cos_mat.flatten().mean()
# This makes sense.

0.14900964

Baseline topic: 'Political Investigation (Cohen/Virginia)'
For each other topic, we calculate the difference between average bias
We expect that the mean difference of bias between this topic and Sports should be POSITIVE and Statistically Significant:

In [63]:
dfs_noNER_descr = pd.read_csv('data/processed/2019_04_30_CNNvsOthersDistanceSummary.csv')

In [72]:
means_noNER = dfs_noNER.groupby(['target_article_category', 'topic']).describe()['cos_similarity']['mean']
means_noNER

target_article_category  topic
bbc                      0        0.138262
                         1        0.169127
                         2        0.151207
                         3        0.085444
                         4        0.158362
                         5        0.149009
                         6        0.186615
                         7        0.161782
                         8        0.089461
                         9        0.157267
                         10       0.114493
                         11       0.142463
                         12       0.152829
                         13       0.184375
                         14       0.146681
                         15       0.176612
                         16       0.143880
                         17       0.135338
                         18       0.172837
                         19       0.210490
                         20       0.145195
                         21       0.108983
                       

In [70]:
# topic_10_df = topic_modeling_df_dropped[topic_modeling_df_dropped.Dominant_Topic == 10]
# topic_10_df_cnn = topic_10_df[topic_10_df.source == 'cnn']
# topic_10_df_bbc = topic_10_df[topic_10_df.source == 'bbc']
# topic_10_df_fox = topic_10_df[topic_10_df.source == 'fox']

# Political Investigation (Cohen/Virginia)
mean_bias_bbc_topic2 = means_noNER['bbc'][2]
mean_bias_fox_topic2 = means_noNER['fox'][2]

# US Campaign Politics:
mean_bias_bbc_topic10 = means_noNER['bbc'][10]
mean_bias_fox_topic10 = means_noNER['fox'][10]

# Health:
mean_bias_bbc_topic7 = means_noNER['bbc'][7]
mean_bias_fox_topic7 = means_noNER['fox'][7]

# International Affairs (Middle East and South Asia):
mean_bias_bbc_topic16 = means_noNER['bbc'][16]
mean_bias_fox_topic16 = means_noNER['fox'][16]

# Sports:
mean_bias_bbc_topic1 = means_noNER['bbc'][1]
mean_bias_fox_topic1 = means_noNER['fox'][1]

# Mean calculations:
# For Political Investigation (Cohen/Virginia) baseline:
mean_diff_bbc_topic7 = mean_bias_bbc_topic2 - mean_bias_bbc_topic7
mean_diff_fox_topic7 = mean_bias_fox_topic2 - mean_bias_fox_topic7
mean_diff_bbc_topic16 = mean_bias_bbc_topic2 - mean_bias_bbc_topic16
mean_diff_fox_topic16 = mean_bias_fox_topic2 - mean_bias_fox_topic16
mean_diff_bbc_topic1 = mean_bias_bbc_topic2 - mean_bias_bbc_topic1
mean_diff_fox_topic1 = mean_bias_fox_topic2 - mean_bias_fox_topic1

# For US Campaign Politics baseline:
mean_diff_bbc_topic10_7 = mean_bias_bbc_topic10 - mean_bias_bbc_topic1
mean_diff_fox_topic10_7 = mean_bias_fox_topic10 - mean_bias_fox_topic1

print('BASELINE: Political Investigation (Cohen/Virginia):')
print('Mean Difference vs BBC b/w Political Investigation (Cohen/Virginia) AND Health', mean_diff_bbc_topic7)
print('Mean Difference vs FOX b/w Political Investigation (Cohen/Virginia) AND Health', mean_diff_fox_topic7)
print('Mean Difference vs BBC b/w Political Investigation (Cohen/Virginia) AND International Affairs (Middle East and South Asia)', mean_diff_bbc_topic16)
print('Mean Difference vs FOX b/w Political Investigation (Cohen/Virginia) AND International Affairs (Middle East and South Asia)', mean_diff_fox_topic16)
print('Mean Difference vs BBC b/w Political Investigation (Cohen/Virginia) AND Sports', mean_diff_bbc_topic1)
print('Mean Difference vs FOX b/w Political Investigation (Cohen/Virginia) AND Sports', mean_diff_fox_topic1)
print()
print('BASELINE: US Campaign Politics:')
print('Mean Difference vs BBC b/w US Campaign Politics AND Sports', mean_diff_bbc_topic10_7)
print('Mean Difference vs FOX b/w US Campaign Politics AND Sports', mean_diff_fox_topic10_7)

BASELINE: Political Investigation (Cohen/Virginia):
Mean Difference vs BBC b/w Political Investigation (Cohen/Virginia) AND Health -0.010575130581855774
Mean Difference vs FOX b/w Political Investigation (Cohen/Virginia) AND Health -0.01473166048526764
Mean Difference vs BBC b/w Political Investigation (Cohen/Virginia) AND International Affairs (Middle East and South Asia) 0.007326915860176086
Mean Difference vs FOX b/w Political Investigation (Cohen/Virginia) AND International Affairs (Middle East and South Asia) -0.01475001871585846
Mean Difference vs BBC b/w Political Investigation (Cohen/Virginia) AND Sports -0.017919614911079407
Mean Difference vs FOX b/w Political Investigation (Cohen/Virginia) AND Sports -0.02770853042602539

BASELINE: US Campaign Politics:
Mean Difference vs BBC b/w US Campaign Politics AND Sports -0.05463398993015289
Mean Difference vs FOX b/w US Campaign Politics AND Sports -0.0718696117401123


Do the same as above, but more automated:

In [73]:
means_noNER

target_article_category  topic
bbc                      0        0.138262
                         1        0.169127
                         2        0.151207
                         3        0.085444
                         4        0.158362
                         5        0.149009
                         6        0.186615
                         7        0.161782
                         8        0.089461
                         9        0.157267
                         10       0.114493
                         11       0.142463
                         12       0.152829
                         13       0.184375
                         14       0.146681
                         15       0.176612
                         16       0.143880
                         17       0.135338
                         18       0.172837
                         19       0.210490
                         20       0.145195
                         21       0.108983
                       

In [57]:
# modified from: https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)
    # calculate the standard deviation based on unequal variances
    s = sqrt((s1 + s2) / 2)
    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

In [289]:
%%time
from statsmodels.stats.weightstats import ttest_ind

baseline_topic = 2
baseline_topic_str = topic_translation[baseline_topic]
baseline_source = 'cnn'
baseline_source_title = baseline_source.upper()
baseline_source_filt = (topic_modeling_df_dropped.source == baseline_source)
baseline_source_df = topic_modeling_df_dropped[baseline_source_filt]
#baseline_source_baseline_topic_df = topic_modeling_df_dropped[baseline_source_topic_filt]
#baseline_source_baseline_topic_idx = baseline_source_baseline_topic_df.index

# For baseline topic (Political Scandal), mean biases are: TODO: THIS METHOD IS SUSPECT
# mean_bias_bbc_topic2 = means_noNER['bbc'][baseline_topic]
# mean_bias_fox_topic2 = means_noNER['fox'][baseline_topic]
# mean_bias_rt_topic2 = means_noNER['rt'][baseline_topic]

# Collect all t-tests into this DF:
ttests = pd.DataFrame(columns=['Baseline Source', 'Target Source', 'Baseline Topic', 
                               'Target Topic', 'Mean Difference',
                               'Welsh\'s t-Statistic', 'Two-Tailed P-Value', 
                               'Satterthwait DoF', 'Cohen\'s d'])
all_target_sources = ['bbc', 'fox', 'rt']
for target_source in all_target_sources:
    target_source_title = target_source.upper()
    source_comparison_str = baseline_source_title + ' vs ' + target_source_title
    
    # TODO: THIS METHOD IS SUSPECT
    #mean_bias_baseline_topic = means_noNER[target_source][baseline_topic]
    
    # Baseline:
    idx1 = topic_modeling_df_dropped[(topic_modeling_df_dropped.source == baseline_source) & (topic_modeling_df_dropped.Dominant_Topic == baseline_topic)].index
    idx2 = topic_modeling_df_dropped[(topic_modeling_df_dropped.source == target_source) & (topic_modeling_df_dropped.Dominant_Topic == baseline_topic)].index
    filtered_baseline_cos_mat = cos_sim_mat[idx1, :][:, idx2]
    filtered_baseline_cos_mat_flat = filtered_baseline_cos_mat.flatten()
    baseline_mean_bias = filtered_baseline_cos_mat_flat.mean()

    target_source_filt = (topic_modeling_df_dropped.source == target_source)
    target_source_df = topic_modeling_df_dropped[target_source_filt]
    
    for a_topic_num in range(23):
        if a_topic_num == baseline_topic:  # Skip baseline comparison to itself
            continue
            
        # TODO: THIS METHOD IS SUSPECT
        #mean_bias_target_topic = means_noNER[target_source][a_topic_num]
        #mean_diff = mean_bias_baseline_topic - mean_bias_target_topic
        
        # Alternative way:
        baseline_topic_filt = (baseline_source_df.Dominant_Topic == a_topic_num)
        baseline_topic_filt_idx = baseline_source_df[baseline_topic_filt].index
        target_topic_filt = (target_source_df.Dominant_Topic == a_topic_num)
        target_topic_filt_idx = target_source_df[target_topic_filt].index
        
        
        filtered_cos_mat = cos_sim_mat[baseline_topic_filt_idx, :][:, target_topic_filt_idx]
        filtered_cos_mat_flat = filtered_cos_mat.flatten()
        
        target_mean_bias = filtered_cos_mat_flat.mean()
        
        diff_in_means = baseline_mean_bias - target_mean_bias
        
        tstat, pvalue, df = ttest_ind(filtered_baseline_cos_mat_flat, 
                                      filtered_cos_mat_flat, 
                                      usevar='unequal')
        cohens_d = cohend(filtered_baseline_cos_mat_flat, filtered_cos_mat_flat)
        a_row = {'Baseline Source': baseline_source_title, 
                 'Target Source': target_source_title, 
                 'Baseline Topic': baseline_topic_str, 
                 'Target Topic': topic_translation[a_topic_num], 
                 'Mean Difference': round(diff_in_means, 6),
                 'Welsh\'s t-Statistic': round(tstat, 6),
                 'Two-Tailed P-Value': round(pvalue, 6),
                 'Satterthwait DoF': round(df, 2),
                 'Cohen\'s d': round(cohens_d, 4)
                }
        ttests = ttests.append(a_row, ignore_index=True)

CPU times: user 4.47 s, sys: 620 ms, total: 5.09 s
Wall time: 1.41 s


In [262]:
idx1 = topic_modeling_df_dropped[(topic_modeling_df_dropped.source == 'cnn') & (topic_modeling_df_dropped.Dominant_Topic == 2)].index

In [263]:
idx2 = topic_modeling_df_dropped[(topic_modeling_df_dropped.source == 'bbc') & (topic_modeling_df_dropped.Dominant_Topic == 2)].index

In [264]:
cos_sim_mat[idx1, :][:, idx2].flatten().mean()

0.15120524

In [281]:
0.05 / 66

0.0007575757575757576

Save, just in case:

In [295]:
ttests.to_csv('data/processed/2019_04_30_ttests.csv', index=False)

In [389]:
ttests = pd.read_csv('data/processed/2019_04_30_ttests.csv')

In [390]:
ttests.head()

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
0,CNN,BBC,Political Investigation (Cohen/Virginia),Unknown,0.012947,143.463212,0.0,548537.03,0.2564
1,CNN,BBC,Political Investigation (Cohen/Virginia),Sports,-0.017921,-28.569813,0.0,15149.28,-0.2881
2,CNN,BBC,Political Investigation (Cohen/Virginia),Military Nuclear Technology,0.065761,21.615786,0.0,167.23,1.5476
3,CNN,BBC,Political Investigation (Cohen/Virginia),Film Industry,-0.007157,-15.426710,0.0,18759.83,-0.1334
4,CNN,BBC,Political Investigation (Cohen/Virginia),Cybersecurity,0.002196,5.865892,0.0,19802.31,0.0464


In [149]:
ttests[ttests['Target Source'] == 'FOX']

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
22,CNN,FOX,Political Investigation (Cohen/Virginia),Unknown,0.004834,52.246821,0.000000,734115.10,0.0870
23,CNN,FOX,Political Investigation (Cohen/Virginia),Sports,-0.027709,-51.809588,0.000000,12430.22,-0.4921
24,CNN,FOX,Political Investigation (Cohen/Virginia),Military Nuclear Technology,0.036530,12.543261,0.000000,431.68,0.6316
25,CNN,FOX,Political Investigation (Cohen/Virginia),Film Industry,-0.017850,-32.576913,0.000000,12892.76,-0.3096
26,CNN,FOX,Political Investigation (Cohen/Virginia),Cybersecurity,-0.007911,-18.564437,0.000000,22584.57,-0.1367
27,CNN,FOX,Political Investigation (Cohen/Virginia),Entertainment Industry,-0.065236,-14.022652,0.000000,395.24,-0.8567
28,CNN,FOX,Political Investigation (Cohen/Virginia),Health,-0.014732,-25.393682,0.000000,9372.36,-0.2690
29,CNN,FOX,Political Investigation (Cohen/Virginia),Brexit,0.058276,31.961173,0.000000,782.13,1.0996
30,CNN,FOX,Political Investigation (Cohen/Virginia),Space Industry,-0.026306,-44.738367,0.000000,12672.63,-0.4396
31,CNN,FOX,Political Investigation (Cohen/Virginia),US Campaign Politics,0.044161,126.224100,0.000000,26464.93,0.8223


In [60]:
abs(ttests['Cohen\'s d']).describe()

count    66.000000
mean      0.461156
std       0.388702
min       0.009200
25%       0.137775
50%       0.368550
75%       0.671875
max       1.694300
Name: Cohen's d, dtype: float64

Plot Cohen's d:

In [61]:
trace1 = go.Histogram(
    x=abs(ttests['Cohen\'s d']), 
    xbins=dict(
        start=0,
        end=2.0,
        size=0.10
    ),
    marker=dict(
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        ),
    opacity=0.75))

graph_data = [trace1]
layout = go.Layout(
    title='Distribution of Effect Size Magnitude (Cohen\'s d)',
    xaxis=dict(
        title='Cohen\'s d',
        dtick=0.10,
    ),
    yaxis=dict(
        title='Count'
    ),
    shapes= [
        # Line Vertical
        {
            'type': 'line',
            'x0': 0.80,
            'y0': 0,
            'x1': 0.80,
            'y1': 12,
            'line': {
                'width': 2,
                'dash': 'dashdot',
            },
        },]
)
fig = go.Figure(data=graph_data, layout=layout)

plotly.offline.iplot(fig, filename='basic histogram')

In [327]:
ttests

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
0,CNN,BBC,Political Investigation (Cohen/Virginia),Unknown,0.012947,143.463212,0.000000,548537.03,0.2564
1,CNN,BBC,Political Investigation (Cohen/Virginia),Sports,-0.017921,-28.569813,0.000000,15149.28,-0.2881
2,CNN,BBC,Political Investigation (Cohen/Virginia),Military Nuclear Technology,0.065761,21.615786,0.000000,167.23,1.5476
3,CNN,BBC,Political Investigation (Cohen/Virginia),Film Industry,-0.007157,-15.426710,0.000000,18759.83,-0.1334
4,CNN,BBC,Political Investigation (Cohen/Virginia),Cybersecurity,0.002196,5.865892,0.000000,19802.31,0.0464
5,CNN,BBC,Political Investigation (Cohen/Virginia),Entertainment Industry,-0.035410,-9.499398,0.000000,395.36,-0.5760
6,CNN,BBC,Political Investigation (Cohen/Virginia),Health,-0.010577,-9.868597,0.000000,2315.13,-0.2188
7,CNN,BBC,Political Investigation (Cohen/Virginia),Brexit,0.061744,99.733987,0.000000,4835.41,1.4124
8,CNN,BBC,Political Investigation (Cohen/Virginia),Space Industry,-0.006062,-10.397575,0.000000,11695.03,-0.1124
9,CNN,BBC,Political Investigation (Cohen/Virginia),US Campaign Politics,0.036713,53.103377,0.000000,4627.22,0.8027


In [156]:
ttests[(abs(ttests['Cohen\'s d']) >= 0.0) & (ttests['Target Source'] == 'FOX')]

,Baseline Source,Target Source,Baseline Topic,Target Topic,Mean Difference,Welsh's t-Statistic,Two-Tailed P-Value,Satterthwait DoF,Cohen's d
22,CNN,FOX,Political Investigation (Cohen/Virginia),Unknown,0.004834,52.246821,0.000000,734115.10,0.0870
23,CNN,FOX,Political Investigation (Cohen/Virginia),Sports,-0.027709,-51.809588,0.000000,12430.22,-0.4921
24,CNN,FOX,Political Investigation (Cohen/Virginia),Military Nuclear Technology,0.036530,12.543261,0.000000,431.68,0.6316
25,CNN,FOX,Political Investigation (Cohen/Virginia),Film Industry,-0.017850,-32.576913,0.000000,12892.76,-0.3096
26,CNN,FOX,Political Investigation (Cohen/Virginia),Cybersecurity,-0.007911,-18.564437,0.000000,22584.57,-0.1367
27,CNN,FOX,Political Investigation (Cohen/Virginia),Entertainment Industry,-0.065236,-14.022652,0.000000,395.24,-0.8567
28,CNN,FOX,Political Investigation (Cohen/Virginia),Health,-0.014732,-25.393682,0.000000,9372.36,-0.2690
29,CNN,FOX,Political Investigation (Cohen/Virginia),Brexit,0.058276,31.961173,0.000000,782.13,1.0996
30,CNN,FOX,Political Investigation (Cohen/Virginia),Space Industry,-0.026306,-44.738367,0.000000,12672.63,-0.4396
31,CNN,FOX,Political Investigation (Cohen/Virginia),US Campaign Politics,0.044161,126.224100,0.000000,26464.93,0.8223


In [188]:
topic_translation

{0: 'Unknown',
 1: 'Sports',
 2: 'Political Investigation (Cohen/Virginia)',
 3: 'Military Nuclear Technology',
 4: 'Film Industry',
 5: 'Cybersecurity',
 6: 'Entertainment Industry',
 7: 'Health',
 8: 'Brexit',
 9: 'Space Industry',
 10: 'US Campaign Politics',
 11: 'Medical Research',
 12: 'Economy',
 13: 'Emergencies',
 14: 'Religious Intolerance',
 15: 'Family Affairs',
 16: 'International Affairs (Middle East and South Asia)',
 17: 'International Affairs (Latin America)',
 18: 'Corporate News',
 19: 'Fashion/Issues of Race',
 20: 'Crime',
 21: 'Social Media',
 22: 'Nature and Environment'}

In [189]:
means_noNER

target_article_category  topic
bbc                      0        0.138262
                         1        0.169127
                         2        0.151207
                         3        0.085444
                         4        0.158362
                         5        0.149009
                         6        0.186615
                         7        0.161782
                         8        0.089461
                         9        0.157267
                         10       0.114493
                         11       0.142463
                         12       0.152829
                         13       0.184375
                         14       0.146681
                         15       0.176612
                         16       0.143880
                         17       0.135338
                         18       0.172837
                         19       0.210490
                         20       0.145195
                         21       0.108983
                       

In [100]:
topic_2_df = topic_modeling_df_dropped[topic_modeling_df_dropped.Dominant_Topic == 2]  # Investigations
topic_7_df = topic_modeling_df_dropped[topic_modeling_df_dropped.Dominant_Topic == 7]  # Health
topic_10_df = topic_modeling_df_dropped[topic_modeling_df_dropped.Dominant_Topic == 10]  # US Campaign Politics
topic_1_df = topic_modeling_df_dropped[topic_modeling_df_dropped.Dominant_Topic == 1]  # Sports
topic_1_df = topic_modeling_df_dropped[topic_modeling_df_dropped.Dominant_Topic == 1]  # Sports


In [103]:
sum(topic_2_df.ner_tokens.str.contains('Mueller'))

75

In [106]:
sum(topic_1_df.ner_tokens.str.contains('Mueller'))

0

In [95]:
topic_2_df.groupby('ner_tokens').count()[topic_2_df.groupby('ner_tokens').count().source > 5]

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,source,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords,Topic,ner_entities
ner_tokens,,,,,,,,,,,,,,,,,,,,,
(),98,98,127,98,98,98,98,98,98,0,...,127,98,98,98,127,127,127,127,127,127
"(California,)",6,6,7,6,6,6,6,6,6,0,...,7,6,6,6,7,7,7,7,7,7
"(Facebook,)",6,6,7,6,6,6,6,6,6,0,...,7,6,6,6,7,7,7,7,7,7
"(Florida,)",4,4,6,4,4,4,4,4,4,0,...,6,4,4,4,6,6,6,6,6,6
"(Jussie Smollett,)",3,3,9,3,3,3,3,3,3,0,...,9,3,3,3,9,9,9,9,9,9
"(Mueller,)",2,2,6,2,2,2,2,2,2,0,...,6,2,2,2,6,6,6,6,6,6
"(R. Kelly,)",1,1,9,1,1,1,1,1,1,0,...,9,1,1,1,9,9,9,9,9,9
"(RT World News,)",7,7,7,7,7,7,7,7,7,0,...,7,7,7,7,7,7,7,7,7,7
"(Roger Stone,)",5,5,10,5,5,5,5,5,5,0,...,10,5,5,5,10,10,10,10,10,10


In [97]:
topic_7_df.groupby('ner_tokens').count()[topic_7_df.groupby('ner_tokens').count().source > 5]

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,source,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords,Topic,ner_entities
ner_tokens,,,,,,,,,,,,,,,,,,,,,
(),36,36,52,36,36,36,36,36,36,0,...,52,36,36,36,52,52,52,52,52,52
"(FDA,)",3,3,6,3,3,3,3,3,3,0,...,6,3,3,3,6,6,6,6,6,6


In [99]:
topic_10_df.groupby('ner_tokens').count()[topic_10_df.groupby('ner_tokens').count().source > 5]

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,source,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords,Topic,ner_entities
ner_tokens,,,,,,,,,,,,,,,,,,,,,
(),3,3,10,3,3,3,3,3,3,0,...,10,3,3,3,10,10,10,10,10,10


In [101]:
topic_1_df.groupby('ner_tokens').count()[topic_1_df.groupby('ner_tokens').count().source > 5]

,source_url,url,title,movies,text,keywords,meta_keywords,tags,authors,publish_date,...,source,source_detail,pull_type,pull_date,clean_text,Dominant_Topic,Topic_Perc_Contrib,Keywords,Topic,ner_entities
ner_tokens,,,,,,,,,,,,,,,,,,,,,
(),11,11,14,11,11,11,11,11,11,0,...,14,11,11,11,14,14,14,14,14,14


In [ ]:
# Political Investigation:
polit_invest_bbc = dfs[(dfs.topic == 2) & (dfs.target_article_category == 'bbc')].cos_similarity
polit_invest_rt = dfs[(dfs.topic == 2) & (dfs.target_article_category == 'rt')].cos_similarity
polit_invest_fox = dfs[(dfs.topic == 2) & (dfs.target_article_category == 'fox')].cos_similarity

# Sports:
sports_bbc = dfs[(dfs.topic == 1) & (dfs.target_article_category == 'bbc')].cos_similarity
sports_rt = dfs[(dfs.topic == 1) & (dfs.target_article_category == 'rt')].cos_similarity
sports_fox = dfs[(dfs.topic == 1) & (dfs.target_article_category == 'fox')].cos_similarity

# Means:
means_noNER


print('Sports vs US Campaign Politics: BBC;', 'EST:', means[1]['bbc'] - means[10]['bbc'], ttest_ind(data1_1, data1_10, alternative='two-sided', usevar='unequal'))
print('Sports vs US Campaign Politics: RT;', means[1]['rt'] - means[10]['rt'], ttest_ind(data2_1, data2_10, alternative='two-sided', usevar='unequal'))
print('Sports vs US Campaign Politics: FOX;', means[1]['fox'] - means[10]['fox'], ttest_ind(data3_2, data3_10, alternative='two-sided', usevar='unequal'))

print('Political Investigation vs US Campaign Politics: BBC', ttest_ind(data1_2, data1_10, alternative='two-sided', usevar='unequal'))
print('Political Investigation vs US Campaign Politics: RT', ttest_ind(data2_2, data2_10, alternative='two-sided', usevar='unequal'))
print('Political Investigation vs US Campaign Politics: FOX', ttest_ind(data3_2, data3_10, alternative='two-sided', usevar='unequal'))


In [286]:
means

topic  target_article_category
0      bbc                        0.138262
       fox                        0.159065
       rt                         0.151197
1      bbc                        0.169127
       fox                        0.191606
       rt                         0.160198
2      bbc                        0.151207
       fox                        0.163898
       rt                         0.155853
3      bbc                        0.085444
       fox                        0.127368
       rt                         0.148380
4      bbc                        0.158362
       fox                        0.181748
       rt                         0.172320
5      bbc                        0.149009
       fox                        0.171809
       rt                         0.165651
6      bbc                        0.186615
       fox                        0.229133
       rt                         0.232860
7      bbc                        0.161782
       fox             

In [281]:
means = dfs_noNER.groupby(['topic', 'target_article_category']).describe()['cos_similarity']['mean']
means

topic  target_article_category
0      bbc                        0.138262
       fox                        0.159065
       rt                         0.151197
1      bbc                        0.169127
       fox                        0.191606
       rt                         0.160198
2      bbc                        0.151207
       fox                        0.163898
       rt                         0.155853
3      bbc                        0.085444
       fox                        0.127368
       rt                         0.148380
4      bbc                        0.158362
       fox                        0.181748
       rt                         0.172320
5      bbc                        0.149009
       fox                        0.171809
       rt                         0.165651
6      bbc                        0.186615
       fox                        0.229133
       rt                         0.232860
7      bbc                        0.161782
       fox             

In [182]:
from statsmodels.stats.weightstats import ttest_ind

In [199]:
# Sports:
data1_1 = dfs[(dfs.topic == 1) & (dfs.target_article_category == 'bbc')].cos_similarity
data2_1 = dfs[(dfs.topic == 1) & (dfs.target_article_category == 'rt')].cos_similarity
data3_1 = dfs[(dfs.topic == 1) & (dfs.target_article_category == 'fox')].cos_similarity

# Political Investigation:
data1_2 = dfs[(dfs.topic == 2) & (dfs.target_article_category == 'bbc')].cos_similarity
data2_2 = dfs[(dfs.topic == 2) & (dfs.target_article_category == 'rt')].cos_similarity
data3_2 = dfs[(dfs.topic == 2) & (dfs.target_article_category == 'fox')].cos_similarity

# US Campaign Politics:
data1_10 = dfs[(dfs.topic == 10) & (dfs.target_article_category == 'bbc')].cos_similarity
data2_10 = dfs[(dfs.topic == 10) & (dfs.target_article_category == 'rt')].cos_similarity
data3_10 = dfs[(dfs.topic == 10) & (dfs.target_article_category == 'fox')].cos_similarity


print('Sports vs US Campaign Politics: BBC;', 'EST:', means[1]['bbc'] - means[10]['bbc'], ttest_ind(data1_1, data1_10, alternative='two-sided', usevar='unequal'))
print('Sports vs US Campaign Politics: RT;', means[1]['rt'] - means[10]['rt'], ttest_ind(data2_1, data2_10, alternative='two-sided', usevar='unequal'))
print('Sports vs US Campaign Politics: FOX;', means[1]['fox'] - means[10]['fox'], ttest_ind(data3_2, data3_10, alternative='two-sided', usevar='unequal'))

print('Political Investigation vs US Campaign Politics: BBC', ttest_ind(data1_2, data1_10, alternative='two-sided', usevar='unequal'))
print('Political Investigation vs US Campaign Politics: RT', ttest_ind(data2_2, data2_10, alternative='two-sided', usevar='unequal'))
print('Political Investigation vs US Campaign Politics: FOX', ttest_ind(data3_2, data3_10, alternative='two-sided', usevar='unequal'))


Sports vs US Campaign Politics: BBC; EST: 0.05463398993015289 (58.94970752529724, 0.0, 12381.164107754137)
Sports vs US Campaign Politics: RT; 0.037731319665908813 (61.44823986056188, 0.0, 21953.14136299349)
Sports vs US Campaign Politics: FOX; 0.0718696117401123 (126.22410033323743, 0.0, 26464.931366006647)
Political Investigation vs US Campaign Politics: BBC (53.103377478138974, 0.0, 4627.221139632502)
Political Investigation vs US Campaign Politics: RT (66.95598073043215, 0.0, 11107.288918945025)
Political Investigation vs US Campaign Politics: FOX (126.22410033323743, 0.0, 26464.931366006647)


In [222]:
topic_modeling_df_dropped.ner_entities = topic_modeling_df_dropped.ner_entities.astype(str)

In [254]:
GPE_filter = topic_modeling_df_dropped.ner_entities.str.contains('GPE')
PERSON_filter = topic_modeling_df_dropped.ner_entities.str.contains('PERSON')
ORG_filter = topic_modeling_df_dropped.ner_entities.str.contains('ORG')
GPE_df = topic_modeling_df_dropped[GPE_filter]
PERSON_df = topic_modeling_df_dropped[PERSON_filter]
ORG_df = topic_modeling_df_dropped[PERSON_filter]

In [266]:
(GPE_df.groupby('source')['title'].count())

source
bbc    1350
cnn    1124
fox    1605
rt     1992
Name: title, dtype: int64

In [267]:
(PERSON_df.groupby('source')['title'].count())

source
bbc    1220
cnn    1382
fox    1332
rt     1487
Name: title, dtype: int64

In [268]:
(ORG_df.groupby('source')['title'].count())

source
bbc    1220
cnn    1382
fox    1332
rt     1487
Name: title, dtype: int64